# <mark> <b> > 2. </b> Pipeline de extracao de dados de documentos </mark>

<b>2_extracao_pipeline_v1.ipynb = 2_root_doc_extract_v1.ipynb</b>    |     Atual notebook com as funçoes para processamento de PDF Pesquisavel e Raster e a criaçao dos Dataframes de forma independente e unica

### Modules & config

In [1]:
import os
import sys
import shutil
import platform
from io import StringIO
from pathlib import Path
from urllib import response

from outlook_msg import Message
import extract_msg
import zipfile
from pyunpack import Archive
import py7zr

import re
from unidecode import unidecode
from unicodedata import normalize
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import PyPDF2

import csv
import json
import pandas as pd

import uuid
import hashlib

import locale
import time, copy
from pytz import timezone
from datetime import datetime, timezone, timedelta

import cv2
import fitz  # Módulo PyMuPDF
from PIL import Image
from PIL import ImageFont
from PIL import Image, ImageDraw
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer, LTChar
import matplotlib.pyplot as plt
from pdf2image import convert_from_path

import pytesseract

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

import logging

# Modulos da solucao
# import modules.extrai_pdf_pesquisavel as Extc
import modules.cronometro as cron
import modules.nova_extracao_pdf_pesquisavel as novaextra 


#### Config - E-mail
# 1. Caminho do arquivo uma mensagem especifica
msg_dir_path = 'pipeline_extracao_documentos/1_emails_documentos_recebidos/11_emails'

# 2. Path para arquivos atachados compactados
msg_attachment_zip = 'pipeline_extracao_documentos/1_emails_documentos_recebidos/13_attachments'


#### Config - messages
# 3. Caminho do arquivo uma mensagem especifica
msg_outros_path = 'pipeline_extracao_documentos/1_emails_documentos_recebidos/12_messages'

# 4. Path para arquivos recebidos manualmente
arquivos_recebidos_path = 'pipeline_extracao_documentos/1_emails_documentos_recebidos/14_documentos_recebidos'


####Config Processamento Pipeline

# 5. Path para documentos para extracao
documentos_extracao_path = "pipeline_extracao_documentos/2_documentos_para_extracao/21_aguardando_processamento"

# 6. Path para gestao de imagens resized
image_resized_path = "pipeline_extracao_documentos/6_geral_administacao/temp_docs/images/processadas"



# 7. path para arquivos json
json_path = "pipeline_extracao_documentos/5_documentos_processados/jsons"

# 7. Path para DFs e CSVs exportados
export_path = "pipeline_extracao_documentos/6_geral_administacao/exports"

# 8. Path para lixeira
root_garbage_path = "pipeline_extracao_documentos/0_lixeira"


#### paths de objetos para criacao/gestao (dicionarios/datasets)
cnae_dict_path = "pipeline_extracao_documentos/6_geral_administacao/datasets/CNAE_X_ITEM_SERVICO_PREFEITURAS.xlsx"


# 12. poppler path
poppler_path = "/home/dani-boy/miniconda3/envs/tables-detr/bin"

# 13. path para config Tesseract
#tessdata_dir_config = '--tessdata-dir "/home/dani-boy/miniconda3/envs/tables-detr/share/tessdata/" --user-patterns "novo_modelo/modelos/user-patterns2.txt" --dpi 600 --oem 3 --psm 6'

#Modelo atual
#tessdata_dir_config = '--tessdata-dir "/home/dani-boy/miniconda3/envs/tables-detr/share/tessdata/" --user-patterns "novo_modelo/modelos/user-patterns2.txt" --dpi 600 --oem 3 --psm 6'

# definindo localizadcao para pt_BR
locale.setlocale(locale.LC_TIME, "pt_BR.utf8")

logging.basicConfig(
    filename='processamentos/log_ocorrencias.log',
    level=logging.INFO, 
    format='%(asctime)s - %(levelname)s - %(message)s', 
    datefmt='%d/%m/%Y %H:%M:%S'
)

logging.info("kernel reiniciado")

## <mark> <b>2.1</b> FunÇoes de Gerais </mark>

In [7]:
# 1. Geraçao do hash do arquivo
def generate_file_hash(file_path):
    # Abre o arquivo em modo de leitura de bytes
    with open(file_path, "rb") as f:
        # Lê o conteúdo do arquivo
        file_data = f.read()
        # Utiliza o algoritmo SHA-256 para gerar o hash
        file_hash = hashlib.sha256(file_data).hexdigest()
    return file_hash

# 2. Geraçao do Unique_id do arquivo
def generate_unique_id():
    return str(uuid.uuid4())

# 3. XXX Busca proximo Batch
def busca_proximo_batch():
    # Abre o arquivo Excel e lê a coluna 'batch'
    df = pd.read_excel("pipeline_extracao_documentos/6_geral_administacao/exports/df_documento_recebido.xlsx", usecols=["batch"])
    # Pega o último valor da coluna 'batch'
    last_value = df.iloc[-1, 0]
    
    # Extraí o número do último batch e adiciona 1 para o próximo
    last_number = int(last_value.split("_")[1])
    next_number = last_number + 1
    
    # Forma o nome do próximo batch
    next_batch = f"Batch_{next_number}"
    
    return next_batch    

# 4. Função para verificar e criar a pasta se não existir
def check_and_create_folder(folder_path):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        
# 7. funçao que MOVE documentos e gera add_log_transaction_entry para df_log_transctions
def move_doc_processed_file(batch_name, src_path, tgt_path):
    
    function = "move_doc_processed_file"
    source_path = src_path
    file = os.path.basename(source_path)
    sub_dir = os.path.join(tgt_path, batch_name)
    destination_path = os.path.join(sub_dir, file)
    document_action = "move_processed_file"
    transaction_detail = (f'document {file} moved by: {function}')
    df_move = pd.DataFrame()
    try:
        document_unique_id = get_document_id_by_file(batch_name, file)
        check_and_create_folder(destination_path)
        shutil.move(source_path, destination_path)
        sucess = True
        move_log = add_log_transaction_entry(document_unique_id, batch_name, file, document_action, src_path, tgt_path, transaction_detail, sucess)
    except Exception as e:
        print(f"Erro ao mover documento: {e}")
        sucess = False
    
    return move_log    

# 8. Função para adicionar um novo registro em df_source
def add_source_entry(batch_name, file_path, file, type, level, parent_unique_id):
    #unique_id = generate_unique_id(type)
    unique_id = generate_unique_id()
    time_now = cron.timenow_pt_BR()   
    file_hash = generate_file_hash(file_path) 
    if level == 1:
        parent_unique_id = unique_id
    data = {
        'Batch': batch_name,
        'Data': time_now,
        'File': file,
        'Type': type,
        'Level': level,
        'Unique_ID': unique_id,
        'Parent_Unique_ID': parent_unique_id,
        'Hash': file_hash,
        'File_Path': file_path
    }
    return data

# 9. Add nova linha para atualizar df_log_transctions
def add_log_transaction_entry(document_unique_id,batch_name, file, document_action, src_path, tgt_path, transaction_detail, sucess=True):

    data_log = {
        'Dt_Time': cron.timenow_pt_BR(),
        'Batch': batch_name,
        'File' : file,
        'Unique_ID': document_unique_id,
        'Action': document_action,
        'Sorce': src_path,
        'Target': tgt_path,
        'Transction_Detail': transaction_detail,
        'Sucess': sucess,    
    }
    
        
    return data_log


# 10. Consulta multiparametros
def filtrar_df(df, **kwargs):
    query = " & ".join(f"{key} == @kwargs['{key}']" for key in kwargs)
    result = df.query(query)
    return result

# 11. Pesquiso Unique_ID por file
def get_document_id_by_file(batch, file):
    
    result = filtrar_df(df_id_relations, Batch=batch, File=file)
    document_unique_id = result['Unique_ID'].values[0]
    
    return document_unique_id


# 12. Busca filhos - simples
def get_children(batch, file_path):
    
    file = os.path.basename(file_path)
    result = filtrar_df(df_id_relations, Batch=batch, File=file)
    document_unique_id = result['Unique_ID'].values[0]
    # Buscando todos dos filhos de um documento
    return filtrar_df(df_id_relations, Batch=batch, Parent_Unique_ID=document_unique_id)


# 13. Busca pai -simples
def get_father(batch, file_path):
    
    file = os.path.basename(file_path)
    result = filtrar_df(df_id_relations, Batch=batch, File=file)
    parent_unique_id = result['Parent_Unique_ID'].values[0]
    # Buscando todos dos filhos de um documento
    return filtrar_df(df_id_relations, Batch=batch, Unique_ID=parent_unique_id)


# 14. Pesquiso pai pelo Unique_ID e trago um dict
def get_father_data_by_children_file(batch, file):
    
    src_result = filtrar_df(df_id_relations, Batch=batch, File=file)
    src_parent_unique_id = src_result['Parent_Unique_ID'].values[0]
    result = filtrar_df(df_id_relations, Batch=batch, Unique_ID=src_parent_unique_id)
    document_batch = result['Batch'].values[0]
    document_data = result['Data'].values[0]
    document_file = result['File'].values[0]
    document_type = result['Type'].values[0]
    document_level = result['Level'].values[0]
    document_unique_id = result['Unique_ID'].values[0]
    document_parent_unique_id = result['Parent_Unique_ID'].values[0]
    document_hash = result['Hash'].values[0]
    document_file_path = result['File_Path'].values[0]
    
    return {
        'Batch': document_batch, 
        'Data': document_data,
        'File' : document_file,
        'Type': document_type,
        'Level': document_level,
        'Unique_ID': document_unique_id,
        'Parent_Unique_ID': document_parent_unique_id,
        'Hash': document_hash,
        'File_Path': document_file_path,
    }


# 15. Pesquiso pai pelo Unique_ID (document_parent_unique_id) e cria DICT
def get_father_by_unique_id(batch, document_parent_unique_id):
    
    result = filtrar_df(df_id_relations, Batch=batch, Unique_ID=document_parent_unique_id)
    document_batch = result['Batch'].values[0]
    document_data = result['Data'].values[0]
    document_file = result['File'].values[0]
    document_type = result['Type'].values[0]
    document_level = result['Level'].values[0]
    document_unique_id = result['Unique_ID'].values[0]
    document_parent_unique_id = result['Parent_Unique_ID'].values[0]
    document_hash = result['Hash'].values[0]
    document_file_path = result['File_Path'].values[0]
    
    return {
        'Batch': document_batch, 
        'Data': document_data,
        'File' : document_file,
        'Type': document_type,
        'Level': document_level,
        'Unique_ID': document_unique_id,
        'Parent_Unique_ID': document_parent_unique_id,
        'Hash': document_hash,
        'File_Path': document_file_path,
    }
    
# 16. Pesquiso pai pelo file do filho e cria DICT
def get_father_data_by_children_file(batch, file):
    
    src_result = filtrar_df(df_id_relations, Batch=batch, File=file)
    src_parent_unique_id = src_result['Parent_Unique_ID'].values[0]
    result = filtrar_df(df_id_relations, Batch=batch, Unique_ID=src_parent_unique_id)
    document_batch = result['Batch'].values[0]
    document_data = result['Data'].values[0]
    document_file = result['File'].values[0]
    document_type = result['Type'].values[0]
    document_level = result['Level'].values[0]
    document_unique_id = result['Unique_ID'].values[0]
    document_parent_unique_id = result['Parent_Unique_ID'].values[0]
    document_hash = result['Hash'].values[0]
    document_file_path = result['File_Path'].values[0]
    
    return {
        'Batch': document_batch, 
        'Data': document_data,
        'File' : document_file,
        'Type': document_type,
        'Level': document_level,
        'Unique_ID': document_unique_id,
        'Parent_Unique_ID': document_parent_unique_id,
        'Hash': document_hash,
        'File_Path': document_file_path,
    }        
        

# 17. Busca o 'Unique_ID' para definir o Parent_Unique_ID sem considerar 'Level'
def get_parent_unique_id(df_id_relations, batch_name, file, type):
    try:
        parent_unique_id = df_id_relations[(df_id_relations['Batch'] == batch_name) & (df_id_relations['File'] == file) & (df_id_relations['Type'] == type)]['Unique_ID'].values[0]
    except IndexError:
        parent_unique_id = None
        print(f"Unique_ID para Batch {batch_name} e type: {type} nao encontrado em df_id_relations.")
    return parent_unique_id


# 18. funcao para trazer somente o 'Unique_ID'
def get_document_unique_id(df_id_relations, batch_name, file, type, level):
    try:
        document_unique_id = df_id_relations[(df_id_relations['Batch'] == batch_name) & (df_id_relations['File'] == file) & (df_id_relations['Type'] == type) & (df_id_relations['Level'] == level)]['Unique_ID'].values[0]
    except IndexError:
        document_unique_id = None
        print(f"Unique_ID para Batch {batch_name} e type: {type} nao encontrado em df_id_relations.")
    return document_unique_id


# 19. funcao para trazer somente o 'Parent_Unique_ID'
def get_document_parent_unique_id(df_id_relations, batch_name, file, type, level):
    try:
        document_parent_unique_id = df_id_relations[(df_id_relations['Batch'] == batch_name) & (df_id_relations['File'] == file) & (df_id_relations['Type'] == type) & (df_id_relations['Level'] == level)]['Parent_Unique_ID'].values[0]
    except IndexError:
        document_parent_unique_id = None
        print(f"Unique_ID para Batch {batch_name} e type: {type} nao encontrado em df_id_relations.")
    return document_parent_unique_id


# 20. funçao para trazer toda a row de df_id_relations para o documento
def get_document_id_relations(df_id_relations, batch_name, file, type, level):
    try:
        document_id_relations = df_id_relations[(df_id_relations['Batch'] == batch_name) & (df_id_relations['File'] == file) & (df_id_relations['Type'] == type) & (df_id_relations['Level'] == level)].values[0]
    except IndexError:
        document_id_relations = None
        print(f"Unique_ID para Batch {batch_name} e type: {type} nao encontrado em df_id_relations.")
    return document_id_relations




# EXEMPLOS de Pesquisa DFss
    # get_document_unique_id(df_id_relations, batch_name, file, type, level)

    # # Busca somente o 'Parent_Unique_ID'
    # get_document_parent_unique_id(df_id_relations, batch_name, file, type, level)


    # #Busca todos os dados da row do documento encontrado
    # document_id_relations = get_document_id_relations(df_id_relations, batch_name, file, type, level)

    # document_batch = document_id_relations[0]
    # document_date = document_id_relations[1]
    # document_name = document_id_relations[2]
    # document_type = document_id_relations[3]
    # document_level = document_id_relations[4]
    # document_unique_id = document_id_relations[5]
    # document_parent_unique_id = document_id_relations[6]
    # document_hash = document_id_relations[7]
    # document_path = document_id_relations[8]

    # # Insercao de um registro pela func add_source_entry
    # file_path = "pipeline_extracao_documentos/1_emails_documentos_recebidos/12_attachments/SPA 15082023.rar"

    # file = os.path.basename(file_path)

    # type = "compressed_file_attachment"

    # level = 1

    # parent_unique_id = ''

    # # Adicionando um novo registro (substitua 'batch_name' e 'email' conforme necessário)
    # new_entry = add_source_entry(batch_name, file_path, file, type, level, parent_unique_id)

    # df_id_relations = df_id_relations.append(new_entry, ignore_index=True)

    # df_id_relations


# Busca proximo Batch caso nao esteja rodando email
batch_name = busca_proximo_batch()

# 1. Criaçao do DataFrame para armazenar as relações de Unique_ID e Parent_Unique_ID
df_id_relations = pd.DataFrame(columns=['Batch', 'Data' ,'File', 'Type', 'Level', 'Unique_ID', 'Parent_Unique_ID', 'Hash', 'File_Path'])

# 2. Criaçao do DataFrame para df_start_pipe:
#df_start_pipe = pd.DataFrame(columns=['Batch', 'Data' ,'File', 'Type', 'Level', 'Unique_ID', 'dt_hora', 'de', 'assunto', 'email', 'Hash'])

def get_template_version(model):
    row_frame = filtrar_df(frames_nf_v4_df, model=model)
    if not row_frame.empty:
            # Acessando a primeira linha do DataFrame filtrado e depois acessando as colunas
            version = [((row_frame.iloc[0]['version']))]
            
    return version[0]  


def filtrar_df(df, **kwargs):
    query = " & ".join(f"{key} == @kwargs['{key}']" for key in kwargs)
    result = df.query(query)
    return result

# 11. Pesquiso Unique_ID por file
def get_document_id_by_file(batch, file):
    
    result = filtrar_df(df_id_relations, Batch=batch, File=file)
    document_unique_id = result['Unique_ID'].values[0]
    
    return document_unique_id


# XXXpara buscar melhor as coordendas dos FRAMES
def get_coordinates_filter(pdf_pesquisavel_map, model, tipo, label, section):
    
    row_frame = filtrar_df(frames_nf_v4_df, model=model, type=tipo, label=label, section_json=section)
    
    # Verificando se row_frame não está vazio
    if not row_frame.empty:
        # Acessando a primeira linha do DataFrame filtrado e depois acessando as colunas
        coodinates = [((row_frame.iloc[0]['x0_p'], row_frame.iloc[0]['y0_p'], row_frame.iloc[0]['x1_p'], row_frame.iloc[0]['y1_p']) if pdf_pesquisavel_map else (row_frame.iloc[0]['x0'], row_frame.iloc[0]['y0'], row_frame.iloc[0]['x1'], row_frame.iloc[0]['y1']))]
    else:
        # Retornando uma tupla de valores NaN se o DataFrame filtrado estiver vazio
        coodinates = [(float('nan'), float('nan'), float('nan'), float('nan'))]
    
    return coodinates









# 2. XXX Analisa nro de paginas
def analisa_nro_pages(file_path):
    
    pdf_document = fitz.open(file_path)
    pages = pdf_document.pages() # generator object

    page_nro = []
    for page in pages:
        page_nro.append(page)
        
    nro_paginas = len(page_nro)    
    if nro_paginas > 1:
        doc_1_page = False
        return doc_1_page, nro_paginas    
    else:
        doc_1_page = True
        return doc_1_page, nro_paginas  
    pdf_document.close()
    
 

# XXX FUNCAO DE SPLIT
def split_documentos(qualquer_df, fase, atividade, status):
    
    documentos_splitados = []
    doc_info = {}
    rows_list = []
    documentos = []
    #output_dir = os.path.join(documentos_scan_path, batch_name)
    num_linhas_df = qualquer_df.shape[0]

    i = num_linhas_df + 1
    for idx, row in qualquer_df.iterrows():
        message_erro = []
        nun_pages = row['pages']
        batch_name = row['batch']
        original_file_name = row['original_file_name']
        folder_name = row['directory']
        file_path = row['file_path']
        level = row['level']
        document_type = row['document_type']
        doc_action = row['doc_action']
        document_unique_id = idx
        new_level = level + 1
        
        if (doc_action == 'splitar') and (status == 'root_analise'):
            if nun_pages > 1:
                try:
                    pdf = fitz.open(file_path)
                    # Número total de páginas no PDF
                    total_pages = len(pdf)
                except Exception as e:
                    print(f"Nao congui abrir o PDF: {e}")    

                # Nome base para os arquivos de saída
                base_name = file_path.split('.')[0]  # Remove a extensão do arquivo
                file_to_delete = file_path
                # Loop para criar um novo PDF para cada página
                for page_num in range(total_pages):
                    # Cria um novo objeto PDF
                    new_pdf = fitz.open()
                    # Adiciona a página atual ao novo PDF
                    new_pdf.insert_pdf(pdf, from_page=page_num, to_page=page_num)
                    # Nome do novo arquivo PDF
                    new_pdf_name = f"{base_name}_page_{page_num + 1}.pdf"
                    # Salva o novo PDF
                    new_pdf.save(new_pdf_name)
                    # Fecha o novo PDF
                    new_pdf.close()
                    rotulo = "prov_nota_fiscal"
                    acao_sugerida = sugestoes_acao.get(rotulo, "no_defined_action")
                    acao_executada = "novo_doc_criado"
                    informations = (f'documento criado a partir do split do documento: {original_file_name}')  
                    name_pdf_splited = os.path.basename(new_pdf_name)

                    new_row = {
                        "seq": i,
                        "date_time": cron.timenow_pt_BR(),
                        "batch": batch_name,
                        "fase_processo": fase,
                        "nome_atividade": atividade,
                        "status_documento": status,
                        "acao_executada": acao_executada,
                        "original_file_name": new_pdf_name,
                        "directory": folder_name,
                        "one_page": True,
                        "pages": 1,
                        "document_type": rotulo,
                        "doc_action": acao_sugerida,
                        "level": level,
                        "document_unique_id": generate_unique_id(),
                        "parent_document_unique_id": document_unique_id,
                        "file_hash": generate_file_hash(file_path),
                        "file_path": file_path,
                        "informations": informations,
                    }
                    rows_list.append(new_row)
                    i += 1
                qualquer_df.loc[idx, 'status_documento'] = "NAO_PROCESSAR" 
                qualquer_df.loc[idx, 'informations'] = "Paginas splitada em multiplos documentos" 
                qualquer_df.loc[idx, 'date_time'] = cron.timenow_pt_BR() 
    
    total_split = i - 1        
    df_split = pd.DataFrame(rows_list)
    
    
    return df_split, rows_list


# XXX Usando na criacao da imagem 
def conv_filename_no_ext(title):
    
    # Divida o título em nome e extensão (mas ignore a extensão)
    name = title.rsplit('.', 1)[0] if '.' in title else title

    # Remova acentos e caracteres especiais do nome
    name = normalize('NFKD', name).encode('ASCII', 'ignore').decode('ASCII')
    
    # Substitua espaços e hífens por sublinhados
    filename = name.replace(' ', '_').replace('-', '_')

    # Remova quaisquer outros caracteres não alfanuméricos, exceto sublinhados
    filename = re.sub(r'[^\w_]', '', filename)

    # Converter para minúsculas
    filename = filename.lower()

    return filename 



def apagar_zone(documentos_extracao_path):
    # Para apagar arquivos PDF:Zone
    for root, dirs, files in os.walk(documentos_extracao_path):
        folder_name = os.path.basename(root)
        for file in files:
            file_path = os.path.join(root, file)
            #print(file)
            if ":Zone" in file:
                file_to_delete = file_path
                os.remove(file_to_delete)
                #print(file, "termina, pode eliminar")
                
                
def confirma_pdf_pequisavel(file_path):
    
    pdf_document = fitz.open(file_path)
    # Página do PDF  ATENCAO  (UNICA PAGINA)
    page_number = 0  # Defina o número da página que deseja analisar
    page = pdf_document[page_number]
    # Definir retângulo de interesse
    try:
        x0 = 0
        y0 = 4
        x1 = 600
        y1 = 200  # Ajuste este valor para delimitar a região vertical
        # Extrair texto dentro do retângulo
        text = page.get_text("text", clip=(x0, y0, x1, y1))
        if text:
            page_number = 0
            pdf_pequisavel = True
        #print(page_number)
        else:
            page_number = 1
            pdf_pequisavel = False
        #print(page_number)
    except Exception as e:
        msg_error = (f"Erro ao abrir pagina do PDF: {e}")
        pdf_pequisavel = False
        pdf_document.close()   
         
        return pdf_pequisavel
                   


def filtrar_df(df, **kwargs):
    query = " & ".join(f"{key} == @kwargs['{key}']" for key in kwargs)
    result = df.query(query)
    return result

# 11. Pesquiso Unique_ID por file
def get_document_id_by_file(batch, file):
    
    result = filtrar_df(df_id_relations, Batch=batch, File=file)
    document_unique_id = result['Unique_ID'].values[0]
    
    return document_unique_id
#generated_parent_document_unique_id = generate_unique_id()  

# Processo de deleçao e atualizacao de documentos
#e_deleta_peloamor(df_docs_splitados)

#me_atualiza_logo_vai_2(novo_df)

# apagar_zone(documentos_extracao_path)


# 5. XXX Ajusta textoYYY
def texto_extraido(texto):
    #0. Tratamento da string
    text_splited = texto.split('\n')
    text_splited = [s.replace(":", "") for s in text_splited]
    text_splited = [x for x in text_splited if x.strip()]
    text_splited = [s.replace(";", "").strip() for s in text_splited] #depende da situaçao
    return text_splited

# 6. XXX Ajusta texto para PDF_Pesquisavel-  NO CABECALHO
def texto_extraido_nf(texto):
    #0. Tratamento da string
    text_splited = texto.split('\n')
    text_splited = [x for x in text_splited if x.strip()]
    text_splited = [s.replace(";", "").strip() for s in text_splited] #depende da situaçao
    return text_splited


# 6. XXX Ajusta texto para PDF RASTER NO CABECALHO
def texto_extraido_cabecalho(texto):
    #0. Tratamento da string
    text_splited = texto.split('\n')
    text_splited = [x for x in text_splited if x.strip()]
    text_splited = [s.replace(";", "").strip() for s in text_splited] #depende da situaçao
    text_splited = [s.replace(")", "").strip() for s in text_splited] #depende da situaçao
    return text_splited
    



#1. funcao: find_value_after_keyword_out_frame_up
def find_value_after_keyword_out_frame_up(keyword, text_list, default_keyword_list=None):
    try:
        index = text_list.index(keyword)
        # Verifica se o valor seguinte não é outra keyword da lista default_keyword_list
        if index + 1 < len(text_list) and text_list[index + 1] not in default_keyword_list:
            return text_list[index + 1]
        else:
            return None

    except ValueError:
        if default_keyword_list:
            for default_keyword in default_keyword_list:
                if default_keyword in text_list:
                    # Caso especial para 'Nome/Razão Social:'
                    if keyword == 'Nome/Razão Social:':
                        return text_list[0]
        return None
    
    
    
    
#2. find_value_after_keyword_out_frame_down  
def find_value_after_keyword_out_frame_down(keyword, text_list, default_keyword_list=None):
    try:
        index = text_list.index(keyword)
        # Verifica se o índice seguinte está dentro da lista
        if index + 1 < len(text_list):
            # Verifica se o valor seguinte não é outra keyword da lista default_keyword_list
            if text_list[index + 1] not in default_keyword_list:
                return text_list[index + 1]
            else:
                return None
        else:
            return None
    except ValueError:
        if default_keyword_list:
            try:
                index = text_list.index(default_keyword_list[-1])
                return text_list[index - 1]
            except ValueError:
                return None
        else:
            return None
        

#3. find_value_after_keyword_fuzz
def find_value_after_keyword_fuzz(keyword, text_list, default_keyword_list=None, fuzziness_threshold=80):
    closest_match = None
    closest_match_score = 0
    
    for i, text in enumerate(text_list):
        score = fuzz.ratio(keyword, text)
        
        if score > closest_match_score:
            closest_match_score = score
            closest_match = text
        
        if closest_match_score > fuzziness_threshold:
            break

    if closest_match_score > fuzziness_threshold:
        index = text_list.index(closest_match)
        if index + 1 < len(text_list):
            if text_list[index + 1] not in default_keyword_list:
                return text_list[index + 1]
            else:
                return None
        else:
            return None
    else:
        return None  




def pesquisa_keyword(string_pesquisa, text_splited, keyword_list):
    resultado_extraido_fuzz = find_value_after_keyword_fuzz(string_pesquisa, text_splited, keyword_list)

    if resultado_extraido_fuzz == None:
        resultado_extraido_frame_up = find_value_after_keyword_out_frame_up(string_pesquisa, text_splited, keyword_list)
        if resultado_extraido_frame_up == None:
            resultado_extraido_frame_down = find_value_after_keyword_out_frame_down(string_pesquisa, text_splited, keyword_list)
            resultado_extraido = resultado_extraido_frame_down
        else:
            resultado_extraido = resultado_extraido_frame_up
    else:
        resultado_extraido = resultado_extraido_fuzz

    # Verifica se o resultado extraído é uma das palavras-chave, indicando um erro
    if resultado_extraido in keyword_list:
        resultado_extraido = None

    return resultado_extraido




# XXX Pequenos mas poderosos
def extract_text_PIL(image, coordinates):
    x0, y0, x1, y1 = coordinates
    image_croped = image.crop((x0, y0, x1, y1))
    texto_extraido = pytesseract.image_to_string(image_croped, lang='por', config='--psm 6')
    return texto_extraido 


# 10. Consulta multiparametros
def filtrar_df(df, **kwargs):
    query = " & ".join(f"{key} == @kwargs['{key}']" for key in kwargs)
    result = df.query(query)
    return result

# 11. Pesquiso Unique_ID por file
def get_document_id_by_file(batch, file):
    
    result = filtrar_df(df_id_relations, Batch=batch, File=file)
    document_unique_id = result['Unique_ID'].values[0]
    
    return document_unique_id


# Funcao importante - process_line
def process_line(value, reference, label):
    name_match = re.search(fr'{reference} (.+)', value)
    if name_match:
        extracted_value = reference + " " + name_match.group(1)
        return {label: extracted_value}
    return None

# Função para extrair número da string
def extract_number(text):
    match = re.search(r'\b\d+(\.\d+)?\b', text)
    if match:
        return match.group(0)
    else:
        return None
    

def corrigir_email(texto):
    # Padrão de regex para identificar e-mails potenciais
    padrao_email = re.compile(r'[a-zA-Z0-9_.+-]+[)!Q@][a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+')
    
    # Encontrar todos os padrões que se assemelham a um e-mail
    possiveis_emails = padrao_email.findall(texto)
    
    for email in possiveis_emails:
        # Se "@" não estiver presente, tentamos corrigir substituindo ")" ou "!" por "@"
        if "@" not in email:
            email_corrigido = email.replace(")", "@").replace("Q", "@")
            texto = texto.replace(email, email_corrigido)
    
    return texto



# 1 XXX Crio o DF  cnae_x_item_servico_df
cnae_x_item_servico_df = pd.read_excel(cnae_dict_path)

# Mapeando prefeitura e CNAE para a descrição do CNAE
cnae_dict = dict(zip(zip(cnae_x_item_servico_df['PREFE'], cnae_x_item_servico_df['CNA_NUMERO']), cnae_x_item_servico_df['CNA_NOME']))

# Mapeando prefeitura e item de serviço para a descrição do item de serviço e o CNAE associado
item_servico_dict = dict(zip(zip(cnae_x_item_servico_df['PREFE'], cnae_x_item_servico_df['ATV_CODIGO']), zip(cnae_x_item_servico_df['ATV_DESCRICAO'], cnae_x_item_servico_df['CNA_NUMERO'])))


# 2. XXX  Tratando o CNAE com dict criado
def processa_cnae_dict(Texto_extraido, de_para_pm, debug):

    text_splited = Texto_extraido.split('\n')
    # Processando CNAE
    cnae_lines = [line for line in text_splited if 'CNAE' in line]

    if cnae_lines:
        cnae_line = cnae_lines[0]
        #print(f'cnae_line: {cnae_line}')
        
        cnae_number = int(extract_number(cnae_line))
        
        cnae_value = cnae_dict.get((de_para_pm, cnae_number),("Valor não encontrado"))
        if cnae_value != "Valor não encontrado":
            cnae_value = cnae_value.upper()
            cnae_value = str(cnae_number) + " - " + cnae_value
            return cnae_value
        else:
            return None
    else:
        cnae_value = processa_cnae_outros(Texto_extraido)
        cnae_number = int(extract_number(cnae_value))

        cnae_value = cnae_dict.get((de_para_pm, cnae_number),("Valor não encontrado"))
        if cnae_value != "Valor não encontrado":
            cnae_value = cnae_value.upper()
            cnae_value = str(cnae_number) + " - " + cnae_value
            return cnae_value
        else:
            return None

     

# 3. XXX  Tratando Item de Servico com dict criado
def processa_itens_servico_dict(Texto_extraido, de_para_pm, debug):
    
    text_splited = Texto_extraido.split('\n')
    # Encontrando a linha que contém o texto desejado
    item_servico_lines = [line for line in text_splited if 'Item da Lista de Serviços' in line]
    #print(f'item_servico_lines (fora do if): {item_servico_lines}')
    # Verificando se encontramos uma linha válida
    if item_servico_lines:
        #print(f'item_servico_lines: {item_servico_lines}')
        item_servico_line = item_servico_lines[0]
        item_servico_cod = float(extract_number(item_servico_line))
        item_servico, cnae_associado = item_servico_dict.get((de_para_pm, item_servico_cod), ("Valor não encontrado", None))
        item_servico = item_servico.upper()
        item_servico_value = str(item_servico_cod) + " - " + item_servico
        return item_servico_value
    
    else:
        #print("Linha com 'Item da Lista de Serviços' não encontrada")
        item_servico_line = processa_item_sevico_outros(Texto_extraido)
        if item_servico_line:
            item_servico_cod = float(extract_number(item_servico_line))
            item_servico, cnae_associado = item_servico_dict.get((de_para_pm, item_servico_cod), ("Valor não encontrado", None))
            item_servico = item_servico.upper()
            item_servico_value = str(item_servico_cod) + " - " + item_servico
            return item_servico_value
        
        else:
            return None
        #return None


def define_rotulo_acao(nome_arquivo, debug):
    
    for palavra_chave, rotulo in mapeamento_palavras_chave.items():
        if palavra_chave.lower() in nome_arquivo.lower():
            break
    else:
        rotulo = 'prov_nota_fiscal' #"sem_rotulo"
        palavra_chave = 'default'
        acao_sugerida = sugestoes_acao.get(rotulo, 'None')
        return palavra_chave, rotulo, acao_sugerida
        # palavra_chave = 'None' #"sem_palavra_chave"
        # acao_sugerida = 'None' #"sem_acao_sugerida"
        
        return palavra_chave, rotulo, acao_sugerida
        #print(f'nome_arquivo: {nome_arquivo} | rotulo: {rotulo}')
    if rotulo != 'None': #"sem_rotulo"
        acao_sugerida = sugestoes_acao.get(rotulo, 'None') # "Ação não definida"
        return palavra_chave, rotulo, acao_sugerida



df_id_relations

,Batch,Data,File,Type,Level,Unique_ID,Parent_Unique_ID,Hash,File_Path


## <mark> <b>2.2</b> FunÇoes de Extracao de dados no Pipeline </mark>

In [8]:
#===========================================================================================#
#                                                                                           #
#                           1. PROCESSAMENTO - PDF PESQUISAVEL                              #
#                                                                                           #   
#===========================================================================================#
# XXXpara buscar melhor as coordendas dos FRAMES
def get_coordinates_filter(pdf_pesquisavel_map, model, tipo, label, section):
    
    row_frame = filtrar_df(frames_nf_v4_df, model=model, type=tipo, label=label, section_json=section)
    
    # Verificando se row_frame não está vazio
    if not row_frame.empty:
        # Acessando a primeira linha do DataFrame filtrado e depois acessando as colunas
        coodinates = [((row_frame.iloc[0]['x0_p'], row_frame.iloc[0]['y0_p'], row_frame.iloc[0]['x1_p'], row_frame.iloc[0]['y1_p']) if pdf_pesquisavel_map else (row_frame.iloc[0]['x0'], row_frame.iloc[0]['y0'], row_frame.iloc[0]['x1'], row_frame.iloc[0]['y1']))]
    else:
        # Retornando uma tupla de valores NaN se o DataFrame filtrado estiver vazio
        coodinates = [(float('nan'), float('nan'), float('nan'), float('nan'))]
    
    return coodinates


# 0.A Dados iniciais - PDF PESQUISAVEL	
def pesquisa_prefeitura_pdf_pesquisavel(idx, row, row_info, map_directory, original_file_name, file_path, debug):    
    
    
   # Carregar o arquivo PDF
    pdf_document = fitz.open(file_path)

    # Página do PDF  ATENCAO  (UNICA PAGINA)
    page_number = 0  # Defina o número da página que deseja analisar
    page = pdf_document[page_number]

    # Definir retângulo de interesse
    x0 = 0
    y0 = 4
    x1 = 600
    y1 = 200  # Ajuste este valor para delimitar a região vertical

    # Extrair texto dentro do retângulo
    text = page.get_text("text", clip=(x0, y0, x1, y1))
    
    if debug:
        print(f'\ndentro da funçao: pesquisa_prefeitura_pdf_pesquisavel: doc.:{original_file_name} | diretorio: {map_directory}  text: \n\n{text}\n\n')
    
    if text:
       page_number = 0
       #print(page_number)
    else:
       page_number = 1
       #print(page_number)
    
    pdf_document.close()
   
    return text

# XXX Funcoes de Regex - cabecalho - documento pdf pesquisavel
nf_data_servico = {}
nf_data_erros = {}
nf_lista_erros = []

# 0. Pesquisa PDF
def is_pdf_searchable(pdf_path):
    try:
        pdf_document = fitz.open(pdf_path)
        is_searchable = all(page.get_text("text") != "" for page in pdf_document)
        pdf_document.close()
        return is_searchable
    except Exception as e:
        print(f"Erro ao verificar o PDF: {e}")
        return False


# 1.A CABECALHO - PDF PESQUISAVEL  
def extrai_cabecalho_PDF_P(idx, row, row_info, section, pdf_pesquisavel_map, de_para_pm, model_map, f_0, f_1, original_file_name, file_path, debug):
    
    nf_data_cabecalho = {}
    lista_erros = []
    label = "1_frame_dados_nf"
    
    batch_name_row_info = row_info.get('batch')
    information_row_info = row_info.get('informations')
    action_item_row_info = row_info.get('action_item')
    
    nf_data_cabecalho['secao'] = section
    nf_data_cabecalho['action_item'] = action_item_row_info
    nf_data_cabecalho['informations'] = information_row_info
    nf_data_cabecalho['processo'] = 'mapeamento regex - PDF pesquisavel'
    
    if debug:
        print(f'\n\n2. dentro da funçao extrai_cabecalho_PDF: batch_name: {batch_name_row_info}\n\n')
    
    pdf_document = fitz.open(file_path)
    page_number = 0  # Defina o número da página que deseja analisar
    page = pdf_document[page_number]    
    tipo = "frame"

    coordinates = get_coordinates_filter(pdf_pesquisavel_map=pdf_pesquisavel_map, model=model_map, tipo=tipo, label=label, section=section)
    x0, y0, x1, y1 = coordinates[0]
    y0 = y0 * f_0
    y1 = y1 * f_1
    
    text = page.get_text("text", clip=(x0, y0, x1, y1))
    if debug:
        print(f'\n3. x0: {x0}, y0: {y0}, x1: {x1}, y1: {y1} f_0: {f_0} f_1: {f_1} | text: \n{text} \n\n')

    try:
        numero_nota_match = re.search(r'Número da Nota:\s+(\d+)', text)
        if numero_nota_match:
            numero_nf = numero_nota_match.group(1)
            nf_data_cabecalho['numero_nota_fiscal'] = numero_nf
            #nf_data_cabecalho['informations'] = 'documento com numero de nota fiscal'
            if debug:
                print(f'\nnr_nro_nf: {nr_nro_nf} - doc: {original_file_name}\n')
        else:
            msg = (f"Número da Nota não encontrado")
            nf_data_cabecalho['numero_nota_fiscal'] = None
            information_row_info = 'Número da Nota não encontrado'
            nf_data_cabecalho['informations'] = information_row_info
            action_item_row_info = 'BREAK_PROCESS'
            nf_data_cabecalho['action_item'] = action_item_row_info
    except Exception as e:
        msg = (f"doc: {original_file_name} | numero NF nao encontrado {e}")
        nf_data_cabecalho['numero_nota_fiscal'] = None
        information_row_info = 'Número da Nota não encontrado'
        nf_data_cabecalho['informations'] = information_row_info
        action_item_row_info = 'BREAK_PROCESS'
        nf_data_cabecalho['action_item'] = action_item_row_info

    # Extrair Competência
    competencia_match = re.search(r'Competência:\s+(.+)', text)
    if competencia_match:
        nf_data_cabecalho['competencia'] = competencia_match.group(1)

    # Extrair Data e Hora de Emissão
    data_emissao_match = re.search(r'Data e Hora da Emissão:\s+(.+)', text)
    if data_emissao_match:
        nf_data_cabecalho['dt_hr_emissao'] = data_emissao_match.group(1)
        
    # Extrair codigo Verificacao
    codigo_verificacao_match = re.search(r'Código Verificação:\s+(.+)', text)
    if codigo_verificacao_match:
        codigo_verificacao_nf = codigo_verificacao_match.group(1)
        nf_data_cabecalho['codigo_verificacao'] =  codigo_verificacao_nf
        tam_codigo_verificacao = len(codigo_verificacao_nf)
        nf_data_cabecalho['conf_cod'] = tam_codigo_verificacao
        
    
    
    pdf_document.close()
    
    return nf_data_cabecalho




# 2.A PRESTADOR DE SERVIÇO - PDF PESQUISAVEL
def extrai_prestador_PDF_P(row, pdf_pesquisavel_map, de_para_pm, model_map, f_0, f_1, original_file_name, file_path, debug):
    
    nf_data_prestador = {}
    section = "2. PRESTADOR DE SERVIÇO"
    pdf_pesquisavel_map = True
    
    label = "2_frame_cnpj_prestador"

    pdf_document = fitz.open(file_path)
    page_number = 0  # Defina o número da página que deseja analisar
    page = pdf_document[page_number]    
    tipo = "frame"

    coordinates = get_coordinates_filter(pdf_pesquisavel_map=pdf_pesquisavel_map, model=model_map, tipo=tipo, label=label, section=section)
    x0, y0, x1, y1 = coordinates[0]
    #print(label)
    #print(x0,x1,y0,y1)
    y0 = y0 * f_0
    y1 = y1 * f_1
    text = page.get_text("text", clip=(x0, y0, x1, y1))
    nf_data_prestador = novaextra.extract_fields_prestador(text)

    pdf_document.close()

    return nf_data_prestador 

  
# 3.A. TOMADOR DE SERVIÇO - PDF PESQUISAVEL
def extrai_tomador_PDF_P(row, pdf_pesquisavel_map, de_para_pm, model_map, f_0, f_1, original_file_name, file_path, debug):

    nf_data_tomador = {}
    section = "3. TOMADOR DE SERVIÇO"
    pdf_pesquisavel_map = True

    label = "3_frame_cnpj_tomador"

    pdf_document = fitz.open(file_path)
    page_number = 0  # Defina o número da página que deseja analisar
    page = pdf_document[page_number]    
    tipo = "frame"

    coordinates = get_coordinates_filter(pdf_pesquisavel_map=pdf_pesquisavel_map, model=model_map, tipo=tipo, label=label, section=section)
    x0, y0, x1, y1 = coordinates[0]
    #print(label)
    #print(x0,x1,y0,y1)
    y0 = y0 * f_0
    y1 = y1 * f_1
    text = page.get_text("text", clip=(x0, y0, x1, y1))
    nf_data_tomador = novaextra.extract_fields_tomador(text)
           

    pdf_document.close()

    return nf_data_tomador 


# 4.A DESCRIMINACAO DOS SERVIÇOS - PDF PESQUISAVEL
def processar_servicos_pdf_pesquisavel(row, pdf_pesquisavel_map, model_map, original_file_name, file_path, debug):
    
    section = "4. DESCRIMINACAO DOS SERVIÇOS"
    frame_type = 'frame'
    f_frame_label = "4_frame_descricao_totais"
    data_box_valores = {'secao': section}
    nf_data_servico = {}
    message_erro = []
    nf_data_servico['original_file_name'] = original_file_name
    pdf_pesquisavel_map = True
    
    filtered_frames_nf_v4_df = frames_nf_v4_df[(frames_nf_v4_df['model'] == model_map) & (frames_nf_v4_df['type'] == frame_type) & (frames_nf_v4_df['label'] == f_frame_label)]
    
    for index_frame, row_frame in filtered_frames_nf_v4_df.iterrows():
        frame_id = row_frame['id']
        label = row_frame['label']
        x0, y0, x1, y1 = (row_frame['x0_p'], row_frame['y0_p'], row_frame['x1_p'], row_frame['y1_p']) if pdf_pesquisavel_map else (row_frame['x0'], row_frame['y0'], row_frame['x1'], row_frame['y1'])
        #print(f'model: {model_map} {"pdf_pesquisavel" if pdf_pesquisavel_map else "raster_pdf"}: {label:>30} | id: {frame_id:>3} | x0: {x0:>6} y0: {y0:>6} x1: {x1:>6} y1: {y1:>6} ')
        
        pdf_document = fitz.open(file_path)
        # Página do PDF
        page_number = 0  # Defina o número da página que deseja analisar
        page = pdf_document[page_number]
        #x0, y0, x1, y1 = (0, 0, 600, 110)
        text = page.get_text("text", clip=(x0, y0, x1, y1))
        
        # Remover quebras de linha e rótulo
        text = text.replace('\n', ' ')
        label = "DISCRIMINAÇÃO DOS SERVIÇOS"
        if text.startswith(label):
            text = text[len(label):].strip()


        try:
            discrimanacao_servico = text   
        except Exception as e:
            msg = (f"doc: | {e}")
            discrimanacao_servico = "Descricao nao encontrada"

        # Atribuir texto ao dicionário
        nf_data_servico['discriminacao_servicos'] = discrimanacao_servico
        
        pdf_document.close()
        
    return nf_data_servico 
            
                
# 5.A VALOR TOTAL - PDF PESQUISAVEL
def processar_valor_total_PDF_P(idx, row, row_info, section, pdf_pesquisavel_map, model_map, original_file_name, file_path, debug):
    
    #nf_data_valor_total = {}
    
    process = ['4_frame_valor_total']
    
    #nf_data_valor_total['secao'] = section
    pdf_pesquisavel_map = True
        
    pdf_document = fitz.open(file_path)
    page_number = 0  # Defina o número da página que deseja analisar
    page = pdf_document[page_number]    
    valor_total_nf = 0.0
        
    tipo= "frame"
    message_erro = []
    #nf_dados_prestador = {}
    # model = row['model']
    for father in process:
        label = father
        if label == "4_frame_valor_total":
            try: 
                #print(model_map)
                coordinates = get_coordinates_filter(pdf_pesquisavel_map=pdf_pesquisavel_map, model=model_map, tipo=tipo, label=label, section=section)
                x0, y0, x1, y1 = coordinates[0]
                text = page.get_text("text", clip=(x0, y0, x1, y1))
                #print(f'model: {model_map} {"pdf_pesquisavel" if pdf_pesquisavel_map else "raster_pdf"}: labe: {label:>30} || x0:{x0:>6} | y0:{y0:>6} | x1:{x1:>6} | y1:{y1:>6}\n {text}')
                valor_total_match = re.search(r'R\$ ([\d,.]+)', text)
                if valor_total_match:
                    valor_total_sem_formatacao = valor_total_match.group(1).replace('.', '').replace(',', '.')
                    if debug:
                        print(f'valor_total_sem_formatacao: {valor_total_sem_formatacao}')
                    valor_total_nf = float(valor_total_sem_formatacao)
                    #nf_data_valor_total['valor_total_nota'] = nf_data_valor_total
                    if debug:
                        print(f'valor_total_nota: {valor_total_nota}')
            except Exception as e:
                msg = (f"doc: | {e}")
                valor_total_nf = 0.0
                # nf_data_valor_total['valor_total_nota'] = 0.0
                logging.error(f" {batch_name} |  doc: {original_file_name:>25} | setion:{section:20} | item: {key:>20} | erro na extracaçao | file_path: {file_path:>40} ")
                
    pdf_document.close()            
                
    return valor_total_nf  


# 6.A CNAE e Item da Lista de Serviços
def extrai_consiste_cnae_PDF_P(idx, row, row_info, section, pdf_pesquisavel_map, de_para_pm, model_map, f_0_cnae, f_1_cnae, f_0_it, f_1_it, original_file_name, file_path, debug):

    nf_data_CNAE = {}
    message_erro = []
    cnae_value = None
    item_servico_value = None
    
    nf_data_CNAE['Secao'] = section
    nf_data_CNAE['texto_cnae'] = None
    nf_data_CNAE['texto_item_servico'] = None
    nf_data_CNAE['original_file_name'] = original_file_name
    
    process = ["cnae", 'item_lista_servicos']
    pdf_pesquisavel_map = True
    
    pdf_document = fitz.open(file_path)
    page_number = 0  # Defina o número da página que deseja analisar
    page = pdf_document[page_number]    
    tipo = "sframe_field"
    
    #print(f'\n item: {original_file_name}\n')
    # model = row['model']
    for father in process:
        cnae_value = None
        item_servico_value = None
        label = father
        if label == "cnae": 
            tipo = "sframe_field"
            coordinates = get_coordinates_filter(pdf_pesquisavel_map=pdf_pesquisavel_map, model=model_map, tipo=tipo, label=label, section=section)
            x0, y0, x1, y1 = coordinates[0]
            #print(f'1.A TRATAMENTO CNAE : (coordenadas originais):  x0:{x0} y0:{y0} x1:{x1} y1:{y1}')
            #print(f'1. label: {label} | coordenadas originais: x0:{x0} y0:{y0} x1:{x1} y1:{y1} item:')
            y0 = y0 * f_0_cnae
            y1 = y1 * f_1_cnae
            text = page.get_text("text", clip=(x0, y0, x1, y1))
            nf_data_CNAE['texto_cnae'] = text
            #print(f'1.A TRATAMENTO CNAE - texto extraido em  coordenadas: x0:{x0} y0:{y0} x1:{x1} y1:{y1}: f_0_cnae: {f_0_cnae}, f_1_cnae: {f_1_cnae} TEXT:\n{text}\n')
            
            # if debug:
            print(f'1.A TRATAMENTO CNAE - texto extraido em  coordenadas: x0:{x0} y0:{y0} x1:{x1} y1:{y1}: f_0_cnae: {f_0_cnae}, f_1_cnae: {f_1_cnae} TEXT:\n{text}\n')
            cnae_processado = processa_cnae_dict(text, de_para_pm, debug)
            # if debug:
            print(f'\n1.B CNAE PROCESSADO: {cnae_processado}\n')
            
        elif label == "item_lista_servicos":
            tipo = "sframe_field"
            #print(model_map)
            coordinates = get_coordinates_filter(pdf_pesquisavel_map=pdf_pesquisavel_map, model=model_map, tipo=tipo, label=label, section=section)
            x0, y0, x1, y1 = coordinates[0]
            #print(f'2.A TRATAMENTO ITENS DE SERVICO:(coordenadas originais):  x0:{x0} y0:{y0} x1:{x1} y1:{y1}')
            y0 = y0 * f_0_it
            y1 = y1 * f_1_it
            text = page.get_text("text", clip=(x0, y0, x1, y1))
            nf_data_CNAE['texto_item_servico'] = text
            # if debug:
            print(f'2.A TRATAMENTO ITENS DE SERVICO -  texto extraido em  coordenadas originais: x0:{x0} y0:{y0} x1:{x1} y1:{y1}: f_0_it:{f_0_it}, f_1_it: {f_1_it} TEXT:\n{text}\n')
            #print(text)
            item_servico_processado = processa_itens_servico_dict(text, de_para_pm, debug)
            # if debug:
            print(f' 2.B ITEM DE SERVICO PROCESSADO: {item_servico_processado} \n\n')

    pdf_document.close()
    
    return cnae_processado, item_servico_processado, nf_data_CNAE


# 6.A XXX Funcao generica de extracao - CNAE_ITEM - PDF_PESQUISAVEL
def extracao_documento_CNAE_ITEM_PDF_P(idx, row, row_info, section, mapping_method, context_mapping, pdf_pesquisavel_map, model_map, def_replace, original_file_name, file_path, debug):

    data_box_valores = {}
    data_box_valores['secao'] = section


    # busco coordenadas para o contexto
    if mapping_method == "frame_&_sframe_field":
        tipo_4_coordinates = "frame"
        tipo_4_filter = "sframe_field"
        
    pdf_document = fitz.open(file_path)
    page_number = 0  # Defina o número da página que deseja analisar
    page = pdf_document[page_number]     

    coordinates = get_coordinates_filter_by_context(pdf_pesquisavel_map, model_map, context_mapping, tipo_4_coordinates)
    x0, y0, x1, y1 = coordinates[0]
    print(x0, y0, x1, y1)  

    text = page.get_text("text", clip=(x0, y0, x1, y1))
    text_splited = text.split('\n')
    text_splited = [x for x in text_splited if x.strip()]

    # 8. Efetuo o filtro para a iteracao
    filtered_frame_nf_v4_df = frames_nf_v4_df[(frames_nf_v4_df['model'] == model_map) & (frames_nf_v4_df['context_mapping'] == context_mapping) & (frames_nf_v4_df['type'] == tipo_4_filter)]

    # 9. iter sobre o filtro
    for index_frame, row_frame in filtered_frame_nf_v4_df.iterrows():
        section = row_frame['section_json']
        label = row_frame['label']
        reference = row_frame['reference']
        marcador_inicio = row_frame['marcador_inicio']
        marcador_fim = row_frame['marcador_fim']
        texto_extraido = extrair_texto_entre_marcadores(text_splited, marcador_inicio, marcador_fim)
        if texto_extraido:
            texto_extraido = texto_extraido.replace(marcador_inicio, "").strip()
            data_box_valores[label] = texto_extraido
            # print(texto_extraido)
            # print()
        else:
            texto_completo = " ".join(text_splited)
            texto_pesquisa = encontrar_texto_fuzzy_marcador_inicial(texto_completo, marcador_inicio, marcador_fim) 
            if texto_pesquisa:
                texto_pesquisa = texto_pesquisa.replace(marcador_inicio, "").strip()
                data_box_valores[label] = texto_pesquisa
                # print(texto_pesquisa)
                # print()
            else:
                texto_pesquisa2 = encontrar_texto_fuzzy_2_marcadores(texto_completo, marcador_inicio, marcador_fim)
                if texto_pesquisa2:
                    texto_pesquisa2 = texto_pesquisa2.replace(marcador_inicio, "").strip()
                    data_box_valores[label] = texto_pesquisa2
    
    pdf_document.close()  
          
    return data_box_valores


# 6.A.1 CNAE Outros
def processa_cnae_outros(text):
    
    nf_data_CNAE_match = re.search(r'CNAE\s+(.+)', text)
    if nf_data_CNAE_match:
        try:
            # Remove a primeira ocorrência de "CNAE:"
            nf_data_CNAE_str = re.sub(r'^CNAE - ', '', text, count=1)
            # Remover quebras de linha
            nf_data_CNAE_str = nf_data_CNAE_str.replace('\n', ' ')
            return nf_data_CNAE_str 
        except Exception as e:
            print(f"Erro busca cnae: {e}") 
        
    return None 

# 6.A.1 CNAE e Item da Lista de Serviços
def processa_item_sevico_outros(text):

    nf_item_lista_servicos_match = re.search(r'Item da Lista de Serviços\s+(.+)', text)
    if nf_item_lista_servicos_match:
        try:
            # Remove a primeira ocorrência de "Item de Servico:"
            nf_item_lista_servicos_str = re.sub(r'^Item da Lista de Serviços - ', '', text, count=1) 
            # Remover quebras de linha
            #nf_item_lista_servicos_str = nf_item_lista_servicos_str.replace('\n \n', '')
            nf_item_lista_servicos_str = nf_item_lista_servicos_str.replace('\n', ' ')
            return nf_item_lista_servicos_str
        except Exception as e:
            print(f"Erro busca cnae: {e}")
            
    return None



# 7.A VALORES E IMPOSTOS - PDF Pesquisavel
def extrai_valores_impostos_PDF_P(idx, row, row_info, section, pdf_pesquisavel_map, de_para_pm, model_map, f_0, f_1, original_file_name, file_path, debug):

    nf_data_valores = {}
    
    
    f_frame_label = "5_frame_valores_impostos"
    pdf_pesquisavel_map = True
    
    tipo = "frame"
    message_erro = []
    
    pdf_document = fitz.open(file_path)
    page_number = 0  # Defina o número da página que deseja analisar
    page = pdf_document[page_number] 
    
    coordinates = get_coordinates_filter(pdf_pesquisavel_map=pdf_pesquisavel_map, model=model_map, tipo=tipo, label=f_frame_label, section=section)
    x0, y0, x1, y1 = coordinates[0]
    y0_tpt = y0
    y1_tpt = y1
    y0 = y0 * f_0
    y1 = y1 * f_1
    text = page.get_text("text", clip=(x0, y0, x1, y1))
    #print(f'Importos: labe: {label:>30} | template:  x0:{x0:>6} | y0:{y0_tpt:>6} | x1:{x1:>6} | y1:{y1_tpt:>6} | ajuste: y0: {y0} | y1: {y1} \n {text}')
    
    nf_data_valores = novaextra.extract_fields_impostos(text)
    
  
    
    #print(text)
    
    pdf_document.close()
    
    return  nf_data_valores
    

# 8.A DADOS COMPLEMENTARES - PDF Pesquisavel
def extrai_dados_complementares_PDF_P(idx, row, row_info, section, pdf_pesquisavel_map, de_para_pm, model_map, f_0, f_1, original_file_name, file_path, debug):
    
    
    f_frame_label = "5_frame_dados_complementares"
    pdf_pesquisavel_map = True
    
    tipo = "frame"
    message_erro = []
    
    pdf_document = fitz.open(file_path)
    page_number = 0  # Defina o número da página que deseja analisar
    page = pdf_document[page_number] 
    
    coordinates = get_coordinates_filter(pdf_pesquisavel_map=pdf_pesquisavel_map, model=model_map, tipo=tipo, label=f_frame_label, section=section)
    x0, y0, x1, y1 = coordinates[0]
    y0_tpt = y0
    y1_tpt = y1
    y0 = y0 * f_0
    y1 = y1 * f_1
    text = page.get_text("text", clip=(x0, y0, x1, y1))
    #print(f'Considerar: label: {f_frame_label:>30} | templt.x0:{x0:>6} | y0:{y0_tpt:>6} | x1:{x1:>6} | y1:{y1_tpt:>6} | ajuste: y0: {y0} | y1: {y1} \n {text}')
    
    nf_data_dados_complementares = {}
    nf_data_dados_complementares['secao'] = "8. DADOS COMPLEMENTARES"

    text = re.sub(r'^DADOS COMPLEMENTARES', '', text, count=1)
    if text == "":
        text = "None"
        nf_data_dados_complementares['dados_complementares'] = text.strip()
    else:    
        # Extrair texto dentro do retângulo
        nf_data_dados_complementares['dados_complementares'] = text.strip()
    
    pdf_document.close()
                        
    return nf_data_dados_complementares  


# 9.A OUTRAS INFORMAÇOES / CRITICAS - PDF Pesquisavel 
def extrai_outras_informacoes_PDF_P(row, pdf_pesquisavel_map, de_para_pm, model_map, f_0, f_1, map_original_file_name, file_path):
    
    nf_data_outras_informacoes = {}
    section = "9. OUTRAS INFORMAÇOES / CRITICAS"
    f_frame_label = "5_frame_inf_criticas"
    pdf_pesquisavel_map = True
    
    tipo = "frame"
    message_erro = []
    
    pdf_document = fitz.open(file_path)
    page_number = 0  # Defina o número da página que deseja analisar
    page = pdf_document[page_number] 
    
    coordinates = get_coordinates_filter(pdf_pesquisavel_map=pdf_pesquisavel_map, model=model_map, tipo=tipo, label=f_frame_label, section=section)
    x0, y0, x1, y1 = coordinates[0]
    y0_tpt = y0
    y1_tpt = y1
    y0 = y0 * f_0
    y1 = y1 * f_1
    text = page.get_text("text", clip=(x0, y0, x1, y1))
    #print(f'Considerar: label: {f_frame_label:>30} | templt.x0:{x0:>6} | y0:{y0_tpt:>6} | x1:{x1:>6} | y1:{y1_tpt:>6} | ajuste: y0: {y0} | y1: {y1} \n {text}')
    nf_data_outras_informacoes = novaextra.extract_fields_outras_info(text)
    
    pdf_document.close()
    
    return nf_data_outras_informacoes       

    
 
# 10.A OBSERVACOES  - PDF Pesquisavel
def extrai_outras_informacoes_PDF_P(row, pdf_pesquisavel_map, de_para_pm, model_map, f_0, f_1, original_file_name, file_path):
    
    nf_data_observacao   = {}
    section = "10. OBSERVACOES"
    f_frame_label = "5_frame_observacao"
    pdf_pesquisavel_map = True

    tipo = "frame"
    message_erro = []

    pdf_document = fitz.open(file_path)
    page_number = 0  # Defina o número da página que deseja analisar
    page = pdf_document[page_number] 

    coordinates = get_coordinates_filter(pdf_pesquisavel_map=pdf_pesquisavel_map, model=model_map, tipo=tipo, label=f_frame_label, section=section)
    x0, y0, x1, y1 = coordinates[0]
    y0_tpt = y0
    y1_tpt = y1
    y0 = y0 * f_0
    y1 = y1 * f_1
    
    text = page.get_text("text", clip=(x0, y0, x1, y1))
    #print(f'Considerar: label: {f_frame_label:>30} | templt.x0:{x0:>6} | y0:{y0_tpt:>6} | x1:{x1:>6} | y1:{y1_tpt:>6} | ajuste: y0: {y0} | y1: {y1} \n {text}')
    # Remove a primeira ocorrência de "Observação:"
    text = re.sub(r'^Observação:', '', text, count=1)
    text = text.replace('\n', ' ')
    nf_data_observacao['observacao'] = text.strip()


    pdf_document.close()
    
    return nf_data_observacao  




#===========================================================================================#
#                                                                                           #
#                           2. PROCESSAMENTO - RASTER PDF                                   #
#                                                                                           #   
#===========================================================================================#

# funçao importante para buscar coordenadas do frame em funçao do contexto
def get_coordinates_filter_by_context(pdf_pesquisavel_map, model_map, context_mapping, tipo):
    
    row_frame = filtrar_df(frames_nf_v4_df, model=model_map, context_mapping=context_mapping, type=tipo)
    
    # Verificando se row_frame não está vazio
    if not row_frame.empty:
        # Acessando a primeira linha do DataFrame filtrado e depois acessando as colunas
        coodinates = [((row_frame.iloc[0]['x0_p'], row_frame.iloc[0]['y0_p'], row_frame.iloc[0]['x1_p'], row_frame.iloc[0]['y1_p']) if pdf_pesquisavel_map else (row_frame.iloc[0]['x0'], row_frame.iloc[0]['y0'], row_frame.iloc[0]['x1'], row_frame.iloc[0]['y1']))]
    else:
        # Retornando uma tupla de valores NaN se o DataFrame filtrado estiver vazio
        coodinates = [(float('nan'), float('nan'), float('nan'), float('nan'))]
    
    return coodinates

# REAVALIAR TODAS - USAR get_coordinates_filter()
def get_coordinates_filter_R_PDF(model_map, tipo, label, section):
    
    row_frame = filtrar_df(frames_nf_v4_df, model=model_map, type=tipo, label=label, section_json=section)
    
    # Verificando se row_frame não está vazio
    if not row_frame.empty:
        # Acessando a primeira linha do DataFrame filtrado e depois acessando as colunas
        coodinates = [(row_frame.iloc[0]['x0'], row_frame.iloc[0]['y0'], row_frame.iloc[0]['x1'], row_frame.iloc[0]['y1'])]
    else:
        # Retornando uma tupla de valores NaN se o DataFrame filtrado estiver vazio
        coodinates = [(float('nan'), float('nan'), float('nan'), float('nan'))]
    
    return coodinates


def get_coordinates_filter_pdf_pesquisavel(model_map, tipo, label, section):
    
    row_frame = filtrar_df(frames_nf_v4_df, model=model_map, type=tipo, label=label, section_json=section)
    
    # Verificando se row_frame não está vazio
    if not row_frame.empty:
        # Acessando a primeira linha do DataFrame filtrado e depois acessando as colunas
        coodinates = [(row_frame.iloc[0]['x0_p'], row_frame.iloc[0]['y0_p'], row_frame.iloc[0]['x1_p'], row_frame.iloc[0]['y1_p'])]
    else:
        # Retornando uma tupla de valores NaN se o DataFrame filtrado estiver vazio
        coodinates = [(float('nan'), float('nan'), float('nan'), float('nan'))]
    
    return coodinates


# 0. INFOMACOES INICIAIS - RASTER PDF
def processar_dados_iniciais(idx, row, row_info, section, map_directory, original_file_name, file_path, debug):
    
    # lista_texto_extraido = []

    nf_dados_doc = {}
    nf_dados_doc['secao'] = section
    pdf_pesquisavel = None
    extracted_txt = pesquisa_prefeitura_pdf_pesquisavel(idx, row, row_info, map_directory, original_file_name, file_path, debug)
    if debug:
        print(f'\n1. funcao: processar_dados_iniciais: doc.:{original_file_name} | diretorio: {map_directory} apos funcao: pesquisa_prefeitura_pdf_pesquisavel: extracted_txt:\n{extracted_txt}\n\n')
    
    if extracted_txt:
        pdf_pesquisavel = True
    else:
        pdf_pesquisavel = False 
       
        x0 = 220
        y0 = 0
        x1= 3858
        y1 = 1572
        
        # usando novo processo que gera o arquivo "on the fly" imagem_gray (converte PDF para imagem de tamanho grande (4134, 5846) - torna-a cinza e a salva)
        imagem_gray, image_resized_name = convert_resize_gray(original_file_name, file_path, image_resized_path)
        extracted_txt = extract_text_PIL(imagem_gray, (x0, y0, x1, y1))
        if debug:
            print(f'\n2. funcao: processar_dados_iniciaisdoc.:{original_file_name} | diretorio: {map_directory}  apos : extract_text_PIL: extracted_txt:\n{extracted_txt}\n\n')
    
    nf_dados_doc['file_name'] = original_file_name    
    nf_dados_doc['pdf_pesquisavel'] = pdf_pesquisavel 
    value = {}   
    texto_tratado = texto_extraido(extracted_txt)
    value = define_dados_iniciais(idx, row, row_info, texto_tratado, debug)
    if debug:
        print(f'\n3. funcao: processar_dados_iniciais doc.:{original_file_name} | diretorio: {map_directory} | apos funcao: define_dados_iniciais() value \n{value}\n\n')
    if value:
        nf_dados_doc.update(value)
   


    return nf_dados_doc


# 1.B CABECALHO XXX Funcoes de extracao -cabecalho Raster
def processar_cabecalho_R_PDF(idx, row, row_info, section, mapping_method, context_mapping, pdf_pesquisavel_map, model_map, original_file_name, file_path, debug):
    
    data_box_valores = {}
    data_box_conferencia = {}
    data_box_valores['secao'] = section
    
    batch_name_row_info = row_info.get('batch')
    #status_documento_row_info = row_info.get('status_documento')
    information_row_info = row_info.get('informations')
    action_item_row_info = row_info.get('action_item')
    
    # Busco a imagem np do documento
    image_np_row_info = row_info.get('image_np')
    
    data_box_valores['action_item'] = action_item_row_info
    data_box_valores['informations'] = information_row_info
    data_box_valores['processo'] = context_mapping
    data_box_valores['conf_cod'] = 0


                     
    
    # busco coordenadas para o contexto
    if mapping_method == "frame_&_sframe_field":
        tipo_4_coordinates = "frame"
        tipo_4_filter = "sframe_field"
    
   
    # 2. usando a funcao de extracao de coordenadas por contexto    
    coordinates = get_coordinates_filter_by_context(pdf_pesquisavel_map, model_map, context_mapping, tipo_4_coordinates)
    x0, y0, x1, y1 = coordinates[0]
    x0 = int(x0)
    y0 = int(y0)
    x1 = int(x1)
    y1 = int(y1) 
    # 3. Cropo a imagem - novo modelo
    cropped_image_np = image_np_row_info[y0:y1, x0:x1] # ajustar nos demais
    data_box_conferencia[f'box_{context_mapping}'] = cropped_image_np
    data_box_conferencia[f'coordinates_{context_mapping}'] = coordinates
    # 4. Converto para PIL
    cropped_image_pil = Image.fromarray(cropped_image_np)
    # 6. Executo OCR
    texto_extraido = pytesseract.image_to_string(cropped_image_pil, lang='por')
    # 7. Trato o texto extraido = text_splited
    text_splited = texto_extraido_cabecalho(texto_extraido)
    if debug:
        print()
        plt.imshow(cropped_image_np)
        plt.axis('off')  # Desativa os eixos para uma visualização mais limpa
        plt.show()
        print(f'\ncoordinates {coordinates} - \ntexto_extraido:\n{text_splited}\n')
        
    # 8. Efetuo o filtro para a iteracao
    filtered_frame_nf_v4_df = frames_nf_v4_df[(frames_nf_v4_df['model'] == model_map) & (frames_nf_v4_df['context_mapping'] == context_mapping) & (frames_nf_v4_df['type'] == tipo_4_filter)]
    
    # 9. iter sobre o filtro
    for index_frame, row_frame in filtered_frame_nf_v4_df.iterrows():
        try:
            section = row_frame['section_json']
            label = row_frame['label']
            reference = row_frame['reference']
            string_pesquisa = row_frame['marcador_inicio']  
            keyword_list = ['Número da Nota:', 'Competência:', 'Data e Hora da Emissão:', 'Código Verificação:']
            texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
            data_box_valores[label] = texto
            if debug:
               print(f'\nidx: {index_frame:> 3} | label: {label} |  string_pesquisa:{string_pesquisa} | dentro do try do raster PDF cabecalho - texto: \n{texto}\n\n')
        except Exception as e:
            msg = (f"{e}")
            data_box_conferencia[label] = msg
    

    # Verificações após o loop
    for key, value in data_box_valores.items():
        if key == 'numero_nota_fiscal' and value is None:
            action_item_row_info = 'BREAK_PROCESS'
            information_row_info = 'Número da Nota não encontrado'
            #logging.error(f" {batch_name} |  doc: {original_file_name:>25} | setion:{section:20} | item: {key:>20} | erro na extracaçao | file_path: {file_path:>40} ")  # Ou registre o erro de outra forma que preferir
        
        elif key == 'codigo_verificacao' and value != None:
            codigo_verificacao_nf = value
            tam_codigo_verificacao = len(codigo_verificacao_nf)
            data_box_valores['conf_cod'] = tam_codigo_verificacao
            
        
        elif key != 'numero_nota_fiscal' and value is None:
            logging.error(f" {batch_name_row_info} |  doc: {original_file_name:>25} | setion:{section:20} | item: {key:>20} | erro na extracaçao | file_path: {file_path:>40} ")  # Ou registre o erro de outra forma que preferir

            
      # if value is None:
        #     logging.error(f" {batch_name} |  doc: {original_file_name:>25} | setion:{section:20} | item: {key:>20} | erro na extracaçao | file_path: {file_path:>40} ")  # Ou registre o erro de outra forma que preferir

    data_box_valores['action_item'] = action_item_row_info
    data_box_valores['informations'] = information_row_info

    
    return data_box_valores

 


# 2.B PRESTADOR DE SERVIÇO - RASTER_PDF
def extrai_prestador_R_PDF(idx, row, row_info, pdf_pesquisavel_map, de_para_pm, model_map, f_0, f_1, original_file_name, file_path, debug):
    
    nf_data_prestador = {}
    data_box_conferencia = {}
    dic_erros = {}
    message_erro = []
    section = "2. PRESTADOR DE SERVIÇO"
    pdf_pesquisavel_map = False
    nf_data_prestador['secao'] = "2. PRESTADOR DE SERVIÇO"
    # 1 busco o documento - image_np
    image_np = row_info['image_np']
    batch_name = row_info['batch']

    process = ['2_frame_cnpj_prestador', '2_frame_inscricao_prestador', '2_frame_dados_prestador']
    tipo = "frame"

    # usando novo processo que gera o arquivo "on the fly" imagem_gray (converte PDF para imagem de tamanho grande (4134, 5846) - torna-a cinza e a salva)
    imagem_gray, image_resized_name = convert_resize_gray(original_file_name, file_path, image_resized_path)

    for label in process:
        seq = process.index(label) + 1
        if label == "2_frame_cnpj_prestador":
            coordinates = get_coordinates_filter_R_PDF(model_map, tipo, label, section)
            x0, y0, x1, y1 = coordinates[0]
            x0 = int(x0)
            y0 = int(y0)
            x1 = int(x1)
            y1 = int(y1)
            y0 = y0 * f_0
            y1 = y1 * f_1 
            # 3. Cropo a imagem - novo modelo
            cropped_image_np = image_np[y0:y1, x0:x1]
            data_box_conferencia[f'box_{label}'] = cropped_image_np
            data_box_conferencia[f'coordinates_{label}'] = coordinates
            # 4. Converto para PIL
            cropped_image_pil = Image.fromarray(cropped_image_np)
            # 6. Executo OCR
            texto_extraido = pytesseract.image_to_string(cropped_image_pil, lang='por')
            text_splited = texto_extraido.split('\n')
            if debug:
                plt.imshow(cropped_image_np)
                plt.axis('off')  # Desativa os eixos para uma visualização mais limpa
                plt.show()
                print(f'\ncoordinates {coordinates} - \ntexto_extraido:\n{text_splited}\n')
            #print(f'1. : (coordenadas originais):  x0:{x0} y0:{y0} x1:{x1} y1:{y1}')

            #print(f'2.A   coordenadas ajustadas:   x0:{x0} y0:{y0} x1:{x1} y1:{y1}: f_0:{f_0}, f_1: {f_1}\n')
            # texto_extraido = extract_text_PIL(imagem_gray, (x0, y0, x1, y1))
            
            text_splited = [x for x in text_splited if x.strip()]
            keyword_list = ['CPF/CNPJ:', 'Telefone:']
            string_pesquisa = "CPF/CNPJ:"
            texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
            cpf_cnpj_formatado_match = re.search(r'(\d{2}\.\d{3}\.\d{3}/\d{4}-\d{2})', texto)
            if cpf_cnpj_formatado_match:
                prestador_cpf_cnpj_com_mascara = cpf_cnpj_formatado_match.group(1)
                nf_data_prestador['p_cpf_cnpj_com_mascara'] = prestador_cpf_cnpj_com_mascara
                prestador_cpf_cnpj_sem_mascara = re.sub(r'\D', '', cpf_cnpj_formatado_match.group(1))
                nf_data_prestador['p_cpf_cnpj_sem_mascara'] = prestador_cpf_cnpj_sem_mascara
            else:
                cpf_cnpj_com_mascara = None
                cpf_cnpj_sem_mascara = None
            try:    
                string_pesquisa = "Telefone:"  
                texto_tel = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
                telefone_match = re.search(r'Telefone:\s+([0-9.\s-]+)', texto_tel) 
                if telefone_match: 
                    prestador_telefone_str = telefone_match.group(1)
                else:
                    telefone_match = re.search(r'Telefone:\s+([0-9.\s-]+)', texto)
                    if telefone_match: 
                        prestador_telefone_str = telefone_match.group(1)
                        nf_data_prestador['p_telefone'] = prestador_telefone_str 
                    else:
                        nf_data_prestador['p_telefone'] = texto_tel
            except Exception as e:
                nf_data_prestador['p_telefone'] = None
                new_row = {
                    "row_index": idx,  # Substitua 'index' pela variável que contém o índice da linha atual
                    "erro_inscricao": str(e),
                    "file": original_file_name,
                    "process_label": label
                }              
        
        elif label == "2_frame_inscricao_prestador":
            coordinates = get_coordinates_filter_R_PDF(model_map, tipo, label, section)
            x0, y0, x1, y1 = coordinates[0]
            x0 = int(x0)
            y0 = int(y0)
            x1 = int(x1)
            y1 = int(y1)
            y0 = y0 * f_0
            y1 = y1 * f_1 
            # 3. Cropo a imagem - novo modelo
            cropped_image_np = image_np[y0:y1, x0:x1]
            data_box_conferencia[f'box_{label}'] = cropped_image_np
            data_box_conferencia[f'coordinates_{label}'] = coordinates
            # 4. Converto para PIL
            cropped_image_pil = Image.fromarray(cropped_image_np)
            # 6. Executo OCR
            texto_extraido = pytesseract.image_to_string(cropped_image_pil, lang='por')
            text_splited = texto_extraido.split('\n')
            if debug:
                plt.imshow(cropped_image_np)
                plt.axis('off')  # Desativa os eixos para uma visualização mais limpa
                plt.show()
                print(f'\ncoordinates {coordinates} - \ntexto_extraido:\n{text_splited}\n')
            # y0 = y0 * f_0
            # y1 = y1 * f_1
            # texto_extraido = extract_text_PIL(imagem_gray, (x0, y0, x1, y1))
            # text_splited = texto_extraido.split('\n')
            text_splited = [x for x in text_splited if x.strip()]
            keyword_list = ['Inscrição Municipal:', 'Inscrição Estadual:']
            string_pesquisa = "Inscrição Municipal:"
            prestadpor_inscricao_municipal = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
            nf_data_prestador['p_inscricao_municipal'] = prestadpor_inscricao_municipal
            
            string_pesquisa = "Inscrição Estadual:"
            prestador_inscricao_estadual = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
            nf_data_prestador['p_inscricao_estadual'] = prestador_inscricao_estadual                    

        elif label == "2_frame_dados_prestador":
            coordinates = get_coordinates_filter_R_PDF(model_map, tipo, label, section)
            x0, y0, x1, y1 = coordinates[0]
            x0 = int(x0)
            y0 = int(y0)
            x1 = int(x1)
            y1 = int(y1)
            y0 = y0 * f_0
            y1 = y1 * f_1 
            # 3. Cropo a imagem - novo modelo
            cropped_image_np = image_np[y0:y1, x0:x1]
            data_box_conferencia[f'box_{label}'] = cropped_image_np
            data_box_conferencia[f'coordinates_{label}'] = coordinates
            # 4. Converto para PIL
            cropped_image_pil = Image.fromarray(cropped_image_np)
            # 6. Executo OCR
            texto_extraido = pytesseract.image_to_string(cropped_image_pil, lang='por')
            text_splited = texto_extraido.split('\n')
            if debug:
                plt.imshow(cropped_image_np)
                plt.axis('off')  # Desativa os eixos para uma visualização mais limpa
                plt.show()
                print(f'\ncoordinates {coordinates} - \ntexto_extraido:\n{text_splited}\n')
            #print(f'1. : (coordenadas originais):  x0:{x0} y0:{y0} x1:{x1} y1:{y1}')
            # y0 = y0 * f_0
            # y1 = y1 * f_1
            # #print(f'2.A   coordenadas ajustadas:   x0:{x0} y0:{y0} x1:{x1} y1:{y1}: f_0:{f_0}, f_1: {f_1}\n')
            # texto_extraido = extract_text_PIL(imagem_gray, (x0, y0, x1, y1))
            # text_splited = texto_extraido.split('\n')
            text_splited = [x for x in text_splited if x.strip()]
            
            keyword_list = ['Nome/Razão Social:', 'Nome de Fantasia:', 'Endereço:', 'E-mail:']
            string_pesquisa = "Nome/Razão Social:"
            texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
            prestador_razao_social = texto
            nf_data_prestador['p_razao_social'] = prestador_razao_social

            string_pesquisa = "Nome de Fantasia:"
            texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
            prestador_nome_fantasia = texto
            nf_data_prestador['p_nome_fantasia'] = prestador_nome_fantasia
            
            string_pesquisa = "Endereço:"
            texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
            prestador_endereco = texto
            nf_data_prestador['p_endereco'] = prestador_endereco
            
            string_pesquisa = "E-mail:"
            texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
            prestador_email = texto
            if prestador_email and prestador_email != None:
                prestador_email = corrigir_email(prestador_email)
 
            nf_data_prestador['p_email'] = prestador_email
            
    return nf_data_prestador  

# 3.A. TOMADOR DE SERVIÇO - RASTER PDF
def extrai_tomador_R_PDF(idx, row, pdf_pesquisavel_map, de_para_pm, model_map, f_0, f_1, original_file_name, file_path, debug):
    
    nf_data_tomador = {}
    section = "3. TOMADOR DE SERVIÇO"
    pdf_pesquisavel_map = False
    cpf_cnpj_tomador = {}
    
    dic_erros = {}
    message_erro = []

    nf_data_tomador['secao'] = section

    process = ['3_frame_cnpj_tomador', '3_frame_inscricao_tomador', '3_frame_dados_tomador']
    tipo = "frame"

    # usando novo processo que gera o arquivo "on the fly" imagem_gray (converte PDF para imagem de tamanho grande (4134, 5846) - torna-a cinza e a salva)
    imagem_gray, image_resized_name = convert_resize_gray(original_file_name, file_path, image_resized_path)
    
    i = 1
    for label in process:
        seq = process.index(label) + 1
        if label == "3_frame_cnpj_tomador":
            
            try:
                coordinates = get_coordinates_filter_R_PDF(model_map, tipo, label, section)
                x0, y0, x1, y1 = coordinates[0]
                #print(f'1. : (coordenadas originais):  x0:{x0} y0:{y0} x1:{x1} y1:{y1}')
                y0 = y0 * f_0
                y1 = y1 * f_1
                #print(f'2.A   coordenadas ajustadas:   x0:{x0} y0:{y0} x1:{x1} y1:{y1}: f_0:{f_0}, f_1: {f_1}\n')
                texto_extraido = extract_text_PIL(imagem_gray, (x0, y0, x1, y1))
                texto_tomador_cnpj = texto_extraido
                file_name = row['original_file_name']
                #print(f'\ni = {i} | seq.: {seq} file: {file_name} \ntexto_tomador_cnpj:\n{texto_tomador_cnpj}\n')
                
                text_splited = texto_extraido.split('\n')
                text_splited = [x for x in text_splited if x.strip()]
                keyword_list = ['CPF/CNPJ:', 'Telefone:'] # 'Nome/Razão Social:'
                string_pesquisa = "CPF/CNPJ:"
                texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
                #print(f'texto antes do try: {texto}')
                cpf_cnpj_formatado_match = re.search(r'(\d{2}\.\d{3}\.\d{3}/\d{4}-\d{2})|(\d{3}\.\d{3}\.\d{3}-\d{2})|(\d{11})', texto)
                if cpf_cnpj_formatado_match:
                    texto_cpf_cnpj = cpf_cnpj_formatado_match.group()
                    # print(f'\ntexto_cpf_cnpj: {texto_cpf_cnpj}\n')  # Aqui usamos group() sem argumentos para pegar toda a string que correspondeu
                    cpf_cnpj_tomador['t_cpf_cnpj_com_mascara'] = texto_cpf_cnpj  # Aqui atribuímos a string correspondente diretamente
                    tomador_cpf_cnpj_sem_mascara = re.sub(r'\D', '', texto_cpf_cnpj)  # Aqui removemos todos os caracteres não-dígitos da string correspondente
                    cpf_cnpj_tomador['t_cpf_cnpj_sem_mascara'] = tomador_cpf_cnpj_sem_mascara
                    

                string_pesquisa = "Telefone:"  
                texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
                telefone_match = re.search(r'(\(?\+?[0-9]*\)?[-. \s]?[0-9]+[-. \s]?[0-9]+)', texto)
                if telefone_match:
                    telefone = telefone_match.group(0)
                else:
                    telefone = None

                if telefone and len(re.findall(r'\d', telefone)) >= 8:
                    # A string contém pelo menos 8 dígitos, então assumimos que é um número de telefone válido
                    nf_data_tomador['t_telefone'] = telefone
                else:
                    nf_data_tomador['t_telefone'] = None

            except Exception as e:
                nf_data_tomador['t_telefone'] = None
                new_row = {
                    "row_index": idx,  # Substitua 'index' pela variável que contém o índice da linha atual
                    "erro_inscricao": str(e),
                    "file": original_file_name,
                    "process_label": label
                }              
        
        elif label == "3_frame_inscricao_tomador":
            # valor_cpf_cnpj_apos = nf_data_tomador['t_cpf_cnpj_com_mascara']
            # print(f'valor no elif da inscricao:{valor_cpf_cnpj_apos}\n')
            coordinates = get_coordinates_filter_R_PDF(model_map, tipo, label, section)
            x0, y0, x1, y1 = coordinates[0]
            y0 = y0 * f_0
            y1 = y1 * f_1
            texto_extraido = extract_text_PIL(imagem_gray, (x0, y0, x1, y1))
            texto_tomador_inscricao = texto_extraido
            #print(f'\ni = {i} | seq.: {seq} file: {original_file_name} \ntexto_tomador_cnpj:\n{texto_tomador_inscricao}\n')
            
            text_splited = texto_extraido.split('\n')
            text_splited = [x for x in text_splited if x.strip()]
            keyword_list = ['Inscrição Municipal:', 'RG:', 'Inscrição Estadual:']
            string_pesquisa = "Inscrição Municipal:"
            tomador_inscricao_municipal = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
            nf_data_tomador['t_inscricao_municipal'] = tomador_inscricao_municipal
            
            string_pesquisa = "RG:"
            tomador_rg = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
            nf_data_tomador['t_rg'] = tomador_rg
            
            string_pesquisa = "Inscrição Estadual:"
            tomador_inscricao_estadual = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
            nf_data_tomador['t_inscricao_estadual'] = tomador_inscricao_estadual                    

        elif label == "3_frame_dados_tomador":
            coordinates = get_coordinates_filter_R_PDF(model_map, tipo, label, section)
            x0, y0, x1, y1 = coordinates[0]
            #print(f'1. : (coordenadas originais):  x0:{x0} y0:{y0} x1:{x1} y1:{y1}')
            y0 = y0 * f_0
            y1 = y1 * f_1
            #print(f'2.A   coordenadas ajustadas:   x0:{x0} y0:{y0} x1:{x1} y1:{y1}: f_0:{f_0}, f_1: {f_1}\n')
            texto_extraido = extract_text_PIL(imagem_gray, (x0, y0, x1, y1))
            texto_tomador_dados = texto_extraido
            #print(f'\ni = {i} | seq.: {seq} file: {original_file_name} \ntexto_tomador_cnpj:\n{texto_tomador_dados}\n')
            text_splited = texto_extraido.split('\n')
            text_splited = [x for x in text_splited if x.strip()]
            
            keyword_list = ['Nome/Razão Social:', 'Nome de Fantasia:', 'Endereço:', 'E-mail:']
            string_pesquisa = "Nome/Razão Social:"
            texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
            texto_limpo = texto.replace('“', '').replace('”', '')
            tomador_razao_social = texto_limpo
            nf_data_tomador['t_razao_social'] = tomador_razao_social

            string_pesquisa = "Endereço:"
            texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
            tomador_endereco = texto
            nf_data_tomador['t_endereco'] = tomador_endereco
            
            string_pesquisa = "E-mail:"
            texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
            tomador_email = corrigir_email(texto)
            
            nf_data_tomador['t_email'] = tomador_email
            
            # valor_cpf_cnpj_apos = nf_data_tomador['t_cpf_cnpj_com_mascara']
            # print(f'valor no no fim do bloco:{valor_cpf_cnpj_apos}\n')
            
        i += 1
        #print(nf_data_tomador)
    nf_data_tomador['t_cpf_cnpj_com_mascara'] = cpf_cnpj_tomador['t_cpf_cnpj_com_mascara']
    nf_data_tomador['t_cpf_cnpj_sem_mascara'] = cpf_cnpj_tomador['t_cpf_cnpj_sem_mascara'] 
            
    return nf_data_tomador



# 7.B VALORES E IMPOSTOS - Raster_PDF
def extracao_impostos_R_PDF(section, tipo, father_value, de_para_pm, model_map, original_file_name, file_path):

    data_box_valores = {}
    data_box_valores['secao'] = section
    
    #print(f'father_value: {father_value}, section: {section}, tipo: {tipo}, model_map: {model_map}, de_para_pm: {de_para_pm}, original_file_name: {original_file_name}\n')

    # trato a imagem logo no começo
    imagem_gray, image_resized_name = convert_resize_gray(original_file_name, file_path, image_resized_path)

    # Estabeleco o filtro
    filtered_boxes_info = frames_nf_v4_df[((frames_nf_v4_df['model'] == model_map) & (frames_nf_v4_df['type'] == tipo) & (frames_nf_v4_df['section_json'] == section) & (frames_nf_v4_df['father'] == father_value))]

    for idx_frame, row_frame in filtered_boxes_info.iterrows():
        extracted_text_box = None
        reference = row_frame['reference']
        x0, y0, x1, y1 = row_frame['x0'], row_frame['y0'], row_frame['x1'], row_frame['y1']
        index = idx_frame
        image_gray_croped = imagem_gray.crop((x0, y0, x1, y1))
        extracted_text_box = (pytesseract.image_to_string(image_gray_croped, lang='por'))
        linhas = extracted_text_box.split('\n')
        label = row_frame['label']
        #print(f'para label:{label:>15} e extracted_text_box: {extracted_text_box}  | x0: {x0:>6} | y0: {y0:>6} | x1: {x1:>6} | y1: {y1:>6} |\n\n')
        for texto in linhas:
            valores = re.findall(r'R\$ *([\d\.]+,\d{1,2})|([\d\.]+,\d{1,2})%', texto)
            if 'R$' in texto or ',' in texto:
                # Original format: Remove 'R$', replace dots with nothing, and replace commas with dots
                number_str = texto.replace('R$', '').replace('.', '').replace(',', '.')
                value = float(number_str)
                #print(f'label: {label:>20} | valor: {value}')
                label = row_frame['label']
                data_box_valores[label] = value 
            elif '%' in texto:
                percent_str = texto.replace('%', '')
                value = float(percent_str)  
                #print(f'label: {label:>20} | valor: {value}')
                data_box_valores[label] = value 
                
        label = row_frame['label']
        data_box_valores[label] = value        
           
        
    return data_box_valores

# 8.B DADOS COMPLEMENTARES - Raster_PDF
def extracao_complementares_R_PDF(row, section, tipo, father_value, de_para_pm, model_map, original_file_name, file_path):

    data_box_valores = {}
    data_box_valores['secao'] = section
    
    #print(f'\n0. entrei na func. -  model_map: {model_map} | section: {section} | tipo: {tipo} | father_value: {father_value} n\n')
    imagem_gray, image_resized_name = convert_resize_gray(original_file_name, file_path, image_resized_path)
    filtered_boxes_info = frames_nf_v4_df[((frames_nf_v4_df['model'] == model_map) & (frames_nf_v4_df['type'] == tipo) & (frames_nf_v4_df['father'] == father_value))]
    for idx_frame, row_frame in filtered_boxes_info.iterrows():
        extracted_text_box = None
        father_value = row_frame['father']
        label = row_frame['label']
        x0, y0, x1, y1 = row_frame['x0'], row_frame['y0'], row_frame['x1'], row_frame['y1']
        index = idx_frame
        #print(f'1. idx: {idx_frame} label:{label:>15} \n\n')
        image_gray_croped = imagem_gray.crop((x0, y0, x1, y1))
        extracted_text_box = (pytesseract.image_to_string(image_gray_croped, lang='por'))
        texto = extracted_text_box
        text = re.sub(r'^DADOS COMPLEMENTARES', '', extracted_text_box, count=1)
        if text == '':
            value = None
                # data_box_valores[label] = value

        
    return data_box_valores


# 9.B OUTRAS INFORMAÇOES / CRITICAS - Raster_PDF 
def extracao_inforacoes_criticas_R_PDF(section, tipo, father_value, de_para_pm, model_map, original_file_name, file_path):

    data_box_valores = {}
    data_box_valores['secao'] = section

    #print(f'father_value: {father_value}, section: {section}, tipo: {tipo}, model_map: {model_map}, de_para_pm: {de_para_pm}, original_file_name: {original_file_name}\n')
    # trato a imagem logo no começo
    imagem_gray, image_resized_name = convert_resize_gray(original_file_name, file_path, image_resized_path)

    # Estabeleco o filtro
    filtered_boxes_info = frames_nf_v4_df[((frames_nf_v4_df['model'] == model_map) & (frames_nf_v4_df['type'] == tipo) & (frames_nf_v4_df['section_json'] == section) & (frames_nf_v4_df['father'] == father_value))]

    for idx_frame, row_frame in filtered_boxes_info.iterrows():
        extracted_text_box = None
        reference = row_frame['reference']
        x0, y0, x1, y1 = row_frame['x0'], row_frame['y0'], row_frame['x1'], row_frame['y1']
        index = idx_frame
        
        label = row_frame['label']
        image_gray_croped = imagem_gray.crop((x0, y0, x1, y1))
        extracted_text_box = (pytesseract.image_to_string(image_gray_croped, lang='por'))
        #print(extracted_text_box)
        texto = extracted_text_box
        if label == "exigibilidade_iss":
            texto = extracted_text_box
            linhas = [linha for linha in texto.split('\n') if linha.strip()]
            marcador = "EXIGIBILIDADE ISS"
            value  = encontrar_valor_por_marcador(marcador, linhas)
            data_box_valores[label] = value
            
        if label == "regime_tributacao":
            texto = extracted_text_box
            linhas = [linha for linha in texto.split('\n') if linha.strip()]
            marcador = "REGIME TRIBUTAÇÃO"
            value = encontrar_valor_por_marcador(marcador, linhas)
            data_box_valores[label] = value
            
        if label == "simples_nacional":
            texto = extracted_text_box
            linhas = [linha for linha in texto.split('\n') if linha.strip()]
            marcador = "SIMPLES NACIONAL"
            value = encontrar_valor_por_marcador(marcador, linhas) 
            data_box_valores[label] = value 
        
        if label == "issqn_retido":
            texto = extracted_text_box 
            linhas = [linha for linha in texto.split('\n') if linha.strip()]
            marcador = "ISSQN RETIDO"
            valor = encontrar_valor_por_marcador(marcador, linhas)
            data_box_valores[label] = value 
            
            
        if label == "local_pretacao_servico":
            texto = extracted_text_box
            texto = texto.replace('\n\n', " ").strip()
            linhas = [linha for linha in texto.split('\n') if linha.strip()]
            marcador = "LOCAL. PRESTAÇÃO SERVIÇO"
            value = encontrar_valor_por_marcador(marcador, linhas)
            data_box_valores[label] = value 
            
        if label == "local_incidencia":
            texto = extracted_text_box
            linhas = [linha for linha in texto.split('\n') if linha.strip()] 
            marcador = "LOCAL INCIDÊNCIA"
            value = encontrar_valor_por_marcador(marcador, linhas)  
            data_box_valores[label] = value              

                
        #print(f'label:{label:>25} | value: {value}\n')         
           
        
    return data_box_valores


def extrair_exigibilidade_iss(texto):
    # Lista de possíveis valores para este campo
    possiveis_valores = ["Exigível", "Não Exigível"]
    
    # Encontrando o valor mais semelhante no texto
    valor, score = process.extractOne(texto, possiveis_valores)
    
    # Você pode ajustar o limite de score conforme necessário
    if score > 85:
        return valor
    return None

def extrair_regime_tributacao(texto):
    # Lista de possíveis valores para este campo
    possiveis_valores = [
        "Sociedade Limitada", 
        "Microempresário Individual (MEI)", 
        "Sociedade anônima", 
        "Microempresa municipal", 
        "Microempresário e Empresa de Pequeno Porte (ME EPP)"
    ]
    
    # Encontrando o valor mais semelhante no texto
    valor, score = process.extractOne(texto, possiveis_valores)
    
    # Você pode ajustar o limite de score conforme necessário
    if score > 85:
        return valor
    return None

def extrair_simples_nacional(texto):
    # Verificando se o texto contém 'Sim' ou 'Não'
    if 'Sim' in texto:
        # Tentando extrair o valor percentual
        match = re.search(r'Sim \((.*?)%\)', texto)
        if match:
            return f'Sim ({match.group(1)}%)'
        return 'Sim'
    elif 'Não' in texto:
        return 'Não'
    return None

def extrair_issqn_retido(texto):
    # Verificando se o texto contém 'Sim' ou 'Não'
    if 'Sim' in texto:
        return 'Sim'
    elif 'Não' in texto:
        return 'Não'
    return None

def extrair_local_prestacao(texto):
    # Lista de possíveis valores para este campo
    possiveis_valores = [
        'Magé - RJ', 'São Pedro da Aldeia - RJ', 'Armação dos Búzios - RJ',
        'Cabo Frio - RJ', 'Araruama - RJ', 'Rio de Janeiro - RJ',
        'Mesquita - RJ', 'SAO PEDRO DA ALDEIA/RJ', 'MESQUITA/RJ', 'Macaé - RJ'
    ]
    
    # Encontrando o valor mais semelhante no texto
    valor, score = process.extractOne(texto, possiveis_valores)
    
    # Você pode ajustar o limite de score conforme necessário
    if score > 85:
        return valor
    return None

def encontrar_valor_por_marcador(marcador, lista_strings):
    # Mapeando marcadores para funções de pós-processamento
    funcoes_pos_processamento = {
        "EXIGIBILIDADE ISS": extrair_exigibilidade_iss,
        "REGIME TRIBUTAÇÃO": extrair_regime_tributacao,
        "SIMPLES NACIONAL": extrair_simples_nacional,
        "ISSQN RETIDO": extrair_issqn_retido,
        "LOCAL. PRESTAÇÃO SERVIÇO": extrair_local_prestacao,
        "LOCAL INCIDÊNCIA": extrair_local_prestacao  # Usando a mesma função que 'LOCAL. PRESTAÇÃO SERVIÇO'
    }
    
    # Encontre o índice do marcador na lista de strings
    for i, linha in enumerate(lista_strings):
        if marcador.lower() in linha.lower():
            # Suponha que o valor está na próxima linha
            valor_bruto = lista_strings[i + 1]
            
            # Obtemos a função de pós-processamento correspondente ao marcador
            funcao_pos_processamento = funcoes_pos_processamento.get(marcador)
            
            # Se encontrarmos uma função de pós-processamento correspondente, aplicamo-la ao valor bruto
            if funcao_pos_processamento:
                return funcao_pos_processamento(valor_bruto)
            
            return valor_bruto
    return None



def extracao_complementar_R_PDF(idx, row, guarda_texto_doc, section, tipo, label, de_para_pm, model_map, def_replace, original_file_name, debug):
    
    #print(idx, row, guarda_texto_doc, section, label, de_para_pm, model_map, def_replace, original_file_name)
    map_document_unique_id = idx
    nf_data_dados_complementares = {}
    label = label
    section = section
    model_map = model_map
    tipo = tipo
    #print(f'\nidx: {idx} | label: {label} | section: {section} | model_map: {model_map} | tipo: {tipo} | def_replace: {def_replace} | original_file_name: {original_file_name}\n\n')
    
    if guarda_texto_doc['document_unique_id'] == map_document_unique_id:
        texto_documento_uso = guarda_texto_doc['texto_documento']
        #print(f'\n\ntexto_documento_uso: {texto_documento_uso}')
    if texto_documento_uso:
        # Busco marcadores
        texto_completo = " ".join(texto_documento_uso)
        
        marcador_inicio, marcador_fim, modelo = busca_marcadores(model_map, section, tipo, label)
        #print(marcador_inicio, marcador_fim, modelo)
        
        if modelo == model_map:
            texto_extraido = encontrar_texto(texto_completo, marcador_inicio, marcador_fim)
            
            if def_replace:
                texto_extraido = texto_extraido.replace(marcador_inicio, "").strip()
            
                texto_extraido_strip = texto_extraido.strip()
                #print(f'texto_extraido_strip: {texto_extraido_strip}')
                nf_data_dados_complementares['dados_complementares'] = texto_extraido_strip
                
                return nf_data_dados_complementares
            else:
                #print("texto_extraido", texto_extraido)
                nf_data_dados_complementares['dados_complementares'] = texto_extraido
                
                return nf_data_dados_complementares
         
            
        print("nao achou modelo")
        return None 
            
    else:
        print("nao achou texto")
        return None 
    
    
    
    
def extracao_observacoees_R_PDF(idx, row, guarda_texto_doc, section, tipo, label, de_para_pm, model_map, def_replace, original_file_name, debug):
    
    #print(idx, row, guarda_texto_doc, section, label, de_para_pm, model_map, def_replace, original_file_name)
    map_document_unique_id = idx
    nf_data_observacao = {}
    label = label
    section = section
    model_map = model_map
    tipo = tipo
    #print(f'\nidx: {idx} | label: {label} | section: {section} | model_map: {model_map} | tipo: {tipo} | def_replace: {def_replace} | original_file_name: {original_file_name}\n\n')
    
    if guarda_texto_doc['document_unique_id'] == map_document_unique_id:
        texto_documento_uso = guarda_texto_doc['texto_documento']
        #print(f'\n\ntexto_documento_uso: {texto_documento_uso}')
    if texto_documento_uso:
        # Busco marcadores
        texto_completo = " ".join(texto_documento_uso)
        
        marcador_inicio, marcador_fim, modelo = busca_marcadores(model_map, section, tipo, label)
        #print(marcador_inicio, marcador_fim, modelo)
        
        texto_extraido = encontrar_texto(texto_completo, marcador_inicio, marcador_fim)
        if texto_extraido:
            nf_data_observacao['observacao'] = texto_extraido
            
            return nf_data_observacao
        
        else:
            marcador_fim = None
            texto_extraido = encontrar_texto(texto_completo, marcador_inicio, marcador_fim)
            if texto_extraido:
                nf_data_observacao['observacao'] = texto_extraido
                
                return nf_data_observacao
            else:
                return None
                 
            
        print("nao achou modelo")
        return None 
            
    else:
        print("nao achou texto")
        return None  



# 10.B OBSERVACOES  - Raster_PDF 
def extracao_observacao_R_PDF(row, section, tipo, father_value, de_para_pm, model_map, original_file_name, file_path):

    data_box_valores = {}
    data_box_valores['secao'] = section
    #print(f'\n0. entrei na func. -  model_map: {model_map} | section: {section} | tipo: {tipo} | father_value: {father_value} n\n')
    imagem_gray, image_resized_name = convert_resize_gray(original_file_name, file_path, image_resized_path)
    filtered_boxes_info = frames_nf_v4_df[((frames_nf_v4_df['model'] == model_map) & (frames_nf_v4_df['type'] == tipo) & (frames_nf_v4_df['father'] == father_value))]
    for idx_frame, row_frame in filtered_boxes_info.iterrows():
        extracted_text_box = None
        reference = row_frame['reference']
        father_value = row_frame['father']
        label = row_frame['label']
        x0, y0, x1, y1 = row_frame['x0'], row_frame['y0'], row_frame['x1'], row_frame['y1']
        index = idx_frame
        image_gray_croped = imagem_gray.crop((x0, y0, x1, y1))
        extracted_text_box = (pytesseract.image_to_string(image_gray_croped, lang='por'))
        texto = extracted_text_box
        text = re.sub(r'^Observação:', '', extracted_text_box, count=1)
        value = text.replace('\n', ' ')
        data_box_valores[label] = value
        #print(f'idx: {idx_frame} label:{label:>15}    | x0: {x0:>6} | y0: {y0:>6} | x1: {x1:>6} | y1: {y1:>6} |\n{text}\n')
   
    return data_box_valores


# XXX IMPORTANTE - ESTA E A FUNCAO PARA SER UTILIZADA: POIS CONVERTE PARA CINZA E RESIZE: (4134, 5846)
def convert_resize_gray(original_file_name, file_path, image_resized_path):

    name_image = conv_filename_no_ext(original_file_name)
    image_resized_name = os.path.join(f'{image_resized_path}/{str(name_image)}.jpg')
    pages = convert_from_path(file_path, 500, poppler_path=poppler_path)
    # 4. Verifica se ha mais que uma pagina
    if len(pages) > 1:
        raise ValueError("Erro, documento com mais de uma página")
    else:
        # 5. Iterar pelas páginas e redimensionar
        resized_pages = []
        for page in pages:
            resized_page = page.resize((4134, 5846))
            resized_pages.append(resized_page)
            
    imagem_gray = resized_pages[0].convert('L')
    imagem_gray.save(image_resized_name, 'JPEG')

    return  imagem_gray, image_resized_name




# 1 XXX Extracao de dados do documento todo
def cria_guarda_doc_ref_R_PDF(idx, row, de_para_pm, model_map, original_file_name, file_path, image_resized_path, debug):
    
    guarda_texto_doc = {}
    
    imagem_gray, image_resized_name = convert_resize_gray(original_file_name, file_path, image_resized_path)

    texto = (pytesseract.image_to_string(imagem_gray, lang='por'))
    linhas = [linha for linha in texto.split('\n') if linha.strip()]
    guarda_texto_doc['document_unique_id'] = idx
    guarda_texto_doc['original_file_name'] = original_file_name
    guarda_texto_doc['texto_documento'] = linhas
    
    return guarda_texto_doc, linhas


# 2. XXX FunÇao para pesquisar entre marcadores do texto
def extrair_texto_entre_marcadores(texto, marcador_inicio, marcador_fim):
    try:
        # Encontra os índices dos marcadores de início e fim
        indice_inicio = next(i for i, s in enumerate(texto) if marcador_inicio in s)
        indice_fim = next(i for i, s in enumerate(texto) if marcador_fim in s)

        # Se o marcador de início e fim estão na mesma linha
        if indice_inicio == indice_fim:
            inicio = texto[indice_inicio].find(marcador_inicio) + len(marcador_inicio)
            fim = texto[indice_fim].find(marcador_fim)
            return texto[indice_inicio][inicio:fim].strip()
        else:
            # Extrai e retorna o texto entre os marcadores
            return " ".join(texto[indice_inicio:indice_fim])
    except StopIteration:
        # Retorna None se algum dos marcadores não for encontrado
        return None
    
    
# 5 XXX Busca os marcadores no template
def busca_marcadores(model_map, section, tipo, label):
    
    #print('fantes da query em busca_marcadores: {mpdel_map} | {section} | {tipo} | {label}}')
    row_frame = filtrar_df(frames_nf_v4_df, model=model_map, type=tipo, label=label, section_json=section)
    # Verificando se row_frame não está vazio
    if not row_frame.empty:
        # Acessando a primeira linha do DataFrame filtrado e depois acessando as colunas
        modelo = [(row_frame.iloc[0]['model'])]
        marc_ini = [(row_frame.iloc[0]['marcador_inicio'])]
        marc_fim = [(row_frame.iloc[0]['marcador_fim'])]
        prestador = [(row_frame.iloc[0]['prestador'])]
        coodinates = [(row_frame.iloc[0]['x0'], row_frame.iloc[0]['y0'], row_frame.iloc[0]['x1'], row_frame.iloc[0]['y1'])]
        modelo = [(row_frame.iloc[0]['model'])]
        #print(f'\n\n - Dentro do busca marcadores:  modelo: {modelo}   | label: {label} | marcador_inicio: {marcador_inicio} | marcador_fim: {marcador_fim}')
        
    return marc_ini[0], marc_fim[0], modelo[0]  


# 6 XXX Funcao generica de extracao
def extracao_documento_R_PDF(idx, row, guarda_texto_doc, section, tipo, label, de_para_pm, model_map, def_replace, original_file_name, debug):
    
    #print(idx, row, guarda_texto_doc, section, label, de_para_pm, model_map, def_replace, original_file_name)
    map_document_unique_id = idx
    
    label = label
    section = section
    model_map = model_map
    tipo = tipo
    #print(f'\nidx: {idx} | label: {label} | section: {section} | model_map: {model_map} | tipo: {tipo} | def_replace: {def_replace} | original_file_name: {original_file_name}\n\n')
    
    if guarda_texto_doc['document_unique_id'] == map_document_unique_id:
        texto_documento_uso = guarda_texto_doc['texto_documento']
        #print(f'\n\ntexto_documento_uso: {texto_documento_uso}')
    if texto_documento_uso:
        # Busco marcadores
        marcador_inicio, marcador_fim, modelo = busca_marcadores(model_map, section, tipo, label)
        #print(marcador_inicio, marcador_fim, modelo)
        
        if modelo == model_map:
            texto_extraido = extrair_texto_entre_marcadores(texto_documento_uso, marcador_inicio, marcador_fim)
            
            if def_replace:
                texto_extraido = texto_extraido.replace(marcador_inicio, "").strip()
            
                texto_extraido_strip = texto_extraido.strip()
                #print(f'texto_extraido_strip: {texto_extraido_strip}')
                return texto_extraido_strip
            else:
                #print("texto_extraido", texto_extraido)
                return texto_extraido
         
            
        print("nao achou modelo")
        return None 
            
    else:
        print("nao achou texto")
        return None 
    

# 6.B XXX Funcao generica de extracao - CNAE_ITEM - RASTER PDF
def extracao_documento_CNAE_ITEM_R_PDF(idx, row, row_info, guarda_texto_doc, section, mapping_method, context_mapping, pdf_pesquisavel_map, model_map, def_replace, original_file_name, file_path, debug):

    data_box_valores = {}
    data_box_valores['secao'] = section
    # busco coordenadas para o contexto
    if mapping_method == "frame_&_sframe_field":
        tipo_4_coordinates = "frame"
        tipo_4_filter = "sframe_field"
    
    map_document_unique_id = idx
    
    if guarda_texto_doc['document_unique_id'] == map_document_unique_id:
        texto_documento_uso = guarda_texto_doc['texto_documento']

    # 8. Efetuo o filtro para a iteracao
    filtered_frame_nf_v4_df = frames_nf_v4_df[(frames_nf_v4_df['model'] == model_map) & (frames_nf_v4_df['context_mapping'] == context_mapping) & (frames_nf_v4_df['type'] == tipo_4_filter)]

    # 9. iter sobre o filtro
    for index_frame, row_frame in filtered_frame_nf_v4_df.iterrows():
        section = row_frame['section_json']
        label = row_frame['label']
        reference = row_frame['reference']
        marcador_inicio = row_frame['marcador_inicio']
        marcador_fim = row_frame['marcador_fim']
        texto_extraido = extrair_texto_entre_marcadores(texto_documento_uso, marcador_inicio, marcador_fim)
        if texto_extraido:
            texto_extraido = texto_extraido.replace(marcador_inicio, "").strip()
            data_box_valores[label] = texto_extraido
            # print(texto_extraido)
            # print()
        else:
            texto_completo = " ".join(text_splited)
            texto_pesquisa = encontrar_texto_fuzzy_marcador_inicial(texto_completo, marcador_inicio, marcador_fim) 
            if texto_pesquisa:
                texto_pesquisa = texto_pesquisa.replace(marcador_inicio, "").strip()
                data_box_valores[label] = texto_pesquisa
                # print(texto_pesquisa)
                # print()
            else:
                texto_pesquisa2 = encontrar_texto_fuzzy_2_marcadores(texto_completo, marcador_inicio, marcador_fim)
                if texto_pesquisa2:
                    texto_pesquisa2 = texto_pesquisa2.replace(marcador_inicio, "").strip()
                    data_box_valores[label] = texto_pesquisa2
                    # print(texto_pesquisa2)

    
    
    return data_box_valores

    
 
# VERIFICAR QUAL SERA USADA
def encontrar_valores1(texto, marcador1, marcador2):
    # Encontrando o texto entre os dois marcadores
    resultado = re.search(f'{re.escape(marcador1)}(.*?){re.escape(marcador2)}', texto, re.DOTALL)
    if resultado:
        segmento = resultado.group(1)
        
        # Encontrando todos os valores no formato R$ 9,99
        valores = re.findall(r'R\$ \d+,\d{2}', segmento)
        
        # Encontrando as posições dos valores na string original
        posicoes = [m.start() for m in re.finditer(r'R\$ \d+,\d{2}', segmento)]
        
        # Criando uma lista de tuplas (valor, posição)
        valores_com_posicoes = list(zip(valores, posicoes))
        
        return valores_com_posicoes
    else:
        return None
 
 
    
# VERIFICAR QUAL SERA USADA    
def encontrar_valores2(texto, marcador1, marcador2):
    # Certificando-se de que todos os parâmetros são strings
    if isinstance(texto, list):
        texto = '\n'.join(texto)
    if not isinstance(marcador1, str):
        marcador1 = str(marcador1)
    if not isinstance(marcador2, str):
        marcador2 = str(marcador2)

    # Encontrando o texto entre os dois marcadores
    resultado = re.search(f'{re.escape(marcador1)}(.*?){re.escape(marcador2)}', texto, re.DOTALL)
    if resultado:
        segmento = resultado.group(1)
        
        # Encontrando todos os valores no formato R$ 9,99
        valores = re.findall(r'R\$ \d+,\d{2}', segmento)
        
        # Encontrando as posições dos valores na string original
        posicoes = [m.start() for m in re.finditer(r'R\$ \d+,\d{2}', segmento)]
        
        # Criando uma lista de tuplas (valor, posição)
        valores_com_posicoes = list(zip(valores, posicoes))
        
        return valores_com_posicoes
    else:
        return None
    
    
# VERIFICAR QUAL SERA USADA    
def encontrar_valores3(texto, marcador1, marcador2):
    # Certificando-se de que todos os parâmetros são strings
    if isinstance(texto, list):
        texto = '\n'.join(texto)
    if not isinstance(marcador1, str):
        marcador1 = str(marcador1)
    if not isinstance(marcador2, str):
        marcador2 = str(marcador2)

    # Encontrando o texto entre os dois marcadores
    resultado = re.search(f'{re.escape(marcador1)}(.*?){re.escape(marcador2)}', texto, re.DOTALL)
    if resultado:
        segmento = resultado.group(1)
        
        # Encontrando todos os valores no formato R$ 9,99 e as alíquotas como 3%
        valores = re.findall(r'R\$ \d+,\d{2}|(?:\d+,\d{1,2}|\d+)%', segmento)
        
        # Encontrando as posições dos valores na string original
        posicoes = [m.start() for m in re.finditer(r'R\$ \d+,\d{2}|(?:\d+,\d{1,2}|\d+)%', segmento)]
        
        # Criando uma lista de tuplas (valor, posição)
        valores_com_posicoes = list(zip(valores, posicoes))
        
        return valores_com_posicoes
    else:
        return None
 

def encontrar_texto_fuzzy_2_marcadores(texto, marcador_inicial, marcador_final, limite_score=80):
    # Usar FuzzyWuzzy para encontrar o melhor match para o marcador inicial
    palavras = texto.split()
    melhor_match_inicial, score_inicial = process.extractOne(marcador_inicial, palavras)
    
    if score_inicial < limite_score:
        return None
    
    # Encontrar a posição inicial do melhor match
    inicio = texto.find(melhor_match_inicial)
    #print('inicio: ',inicio)
    
    # Cortar o texto para começar após o marcador inicial
    texto_cortado = texto[inicio + len(melhor_match_inicial):]
    #print('texto_cortado: ', texto_cortado)
    
    # Usar FuzzyWuzzy para encontrar o melhor match para o marcador final
    melhor_match_final, score_final = process.extractOne(marcador_final, palavras)
    #print('melhor_match_final: ', melhor_match_final)
    
    if score_final < limite_score:
        return None
    
    # Encontrar a posição do melhor match
    fim = texto_cortado.find(melhor_match_final)
    #print('fim: ', fim)
    if fim == -1:
        return None
    
    return texto_cortado[:fim].strip()

def encontrar_texto_fuzzy_marcador_inicial(texto, marcador_inicial, marcador_final, limite_score=80):
    # Encontrar a posição inicial do marcador
    inicio = texto.find(marcador_inicial)
    if inicio == -1:
        return None
    
    # Cortar o texto para começar após o marcador inicial
    texto_cortado = texto[inicio + len(marcador_inicial):]
    
    # Usar FuzzyWuzzy para encontrar o melhor match para o marcador final
    palavras = texto_cortado.split()
    melhor_match, score = process.extractOne(marcador_final, palavras)
    
    if score >= limite_score:
        # Encontrar a posição do melhor match
        fim = texto_cortado.find(melhor_match)
        return texto_cortado[:fim].strip()
    else:
        return None

 
 
def encontrar_texto(texto, marcador_inicial, marcador_final=None):
    if marcador_final:
        # Procura pelo texto entre os dois marcadores
        resultado = re.search(f'{re.escape(marcador_inicial)}(.*?){re.escape(marcador_final)}', texto, re.DOTALL)
        if resultado:
            return resultado.group(1).strip()
    else:
        # Procura pelo texto do marcador inicial até o final
        resultado = re.search(f'{re.escape(marcador_inicial)}(.*)', texto, re.DOTALL)
        if resultado:
            return resultado.group(1).strip()

    # Retorna None se não encontrar nada
    return None
       

# 11. Pesquiso Unique_ID por file
def get_document_id_by_file(batch, file):
    result = filtrar_df(df_id_relations, Batch=batch, File=file)
    document_unique_id = result['Unique_ID'].values[0]
    
    return document_unique_id

# VERIFICAR DUPLICIDADE
def texto_extraido(texto):
    #0. Tratamento da string
    text_splited = texto.split('\n')
    text_splited = [s.replace(":", "") for s in text_splited]
    text_splited = [x for x in text_splited if x.strip()]
    text_splited = [s.replace(";", "").strip() for s in text_splited] #depende da situaçao
    return text_splited


def format_number(number_str):
    # Check for percentage and handle it
    if '%' in number_str:
        number_str = number_str.replace('%', '')
        return float(number_str)  # You can multiply by 100 here if needed

    # Check if the string contains "R$" or a comma, indicating the original format
    if 'R$' in number_str or ',' in number_str:
        # Original format: Remove 'R$', replace dots with nothing, and replace commas with dots
        number_str = number_str.replace('R$', '').replace('.', '').replace(',', '.')
    else:
        # New format: Extract only the numeric part using regex
        number_str = re.findall(r'[\d\.]+', number_str)[-1]

    return float(number_str)

# Funçao de formatacao de numeros
def format_number2(number_str):
    number_str = number_str.replace('R$', '').replace('.', '').replace(',', '.')
    if '%' in number_str:
        number_str = number_str.replace('%', '')
        return float(number_str)  # multiplica por 100 para fields %
    return float(number_str)



# Função para extrair número da string
def extract_number(text):
    match = re.search(r'\b\d+(\.\d+)?\b', text)
    if match:
        return match.group(0)
    else:
        return None            

## <mark> <b>2.3</b> Templates e Dics </mark>

In [9]:
nf_model_path = "pipeline_extracao_documentos/6_geral_administacao/modelos/frames_nf_v11.xlsx"

# 11. path para datasets CNAE e Itens de Serviço
nf_datasets_path = "pipeline_extracao_documentos/6_geral_administacao/datasets"


#Le a planilha e cria do DF
frames_nf_v4_df = pd.read_excel(nf_model_path)

# Cria dicionários para armazenar diferentes tipos de elementos do modelo
document_info = frames_nf_v4_df[frames_nf_v4_df['type'] == 'document'].iloc[0]
boundaries_info = frames_nf_v4_df[frames_nf_v4_df['type'] == 'boundaries']
sections_info = frames_nf_v4_df[frames_nf_v4_df['type'] == 'section']
frames_info = frames_nf_v4_df[frames_nf_v4_df['type'] == 'frame']
sframe_fields_info = frames_nf_v4_df[frames_nf_v4_df['type'] == 'sframe_field']
field_boxes_info = frames_nf_v4_df[frames_nf_v4_df['type'] == 'field_box']

ver = get_template_version('MAGE')
print(f'frames_nf_v4_df: {ver}')
print()


def define_dados_iniciais(idx, row, row_info, texto_tratado, debug):
    
    dados_iniciais_nf = {}
    #status_documento_row_info = row_info.get('status_documento')
    action_item_row_info = row_info.get('action_item')
    information_row_info = row_info.get('informations')
    
    dados_iniciais_nf['action_item'] = action_item_row_info
    dados_iniciais_nf['informations'] = information_row_info
   


    prefeitura_encontrada = None
    de_para_encontrado = None

    # 7. ZZZ Dicionário para mapear Prefeitura com sua sigla
    de_para_prefeitura = {
        "PREFEITURA DA CIDADE MAGE": "PM_MAGE",
        "PREFEITURA DA CIDADE DE MAGE": "PM_MAGE",
        "PREFEITURA MUNICIPAL DE MAGE": "PM_MAGE",
        "PREFEITURA MUNICIPAL DE SAO PEDRO DA ALDEIA": "PM_SPA",
        "MUNICIPAL DE SAO PEDRO DA ALDEIA": "PM_SPA",
        "PREFEITURA MUNICIPAL DE SAO PEDRO DA\nALDEIA": "PM_SPA",
        "PREFEITURA MUNICIPAL DE SAO PEDRO DA": "PM_SPA",
        "PREFEITURA MUNICIPAL DE MESQUITA": "PM_MESQUITA",
        "PREFEITURA MUNICIPAL DE DE MESQUITA": "PM_MESQUITA",
        # ... adicione 
    }
    

    templates = {
        ("PM_MAGE", None): "MAGE",
        ("PM_SPA", None): "SPA",
        ("Pague agora com o seu Pix", None): "NAO_PROCESSAR",
        # ... adicione outras combinações aqui
    }

    cnpj_encontrado = None
    # Verifique cada linha do texto
    for linha in texto_tratado:
        for pref in de_para_prefeitura.keys():
            if pref in linha:
                #print(linha)
                prefeitura_encontrada = pref
                dados_iniciais_nf['prefeitura'] = prefeitura_encontrada
                if debug:
                    print(f'\n4.funcao: define_dados_iniciais(texto_tratado) - dentro do loop for de pesquisa prefeitura - prefeitura_encontrada: \n{prefeitura_encontrada}\n\n')
    # Saímos do loop, agora vamos verificar qual template usar
    if prefeitura_encontrada:
        de_para_pm = de_para_prefeitura.get(prefeitura_encontrada)
        dados_iniciais_nf['de_para_pm'] = de_para_pm
        if debug:
            print(f'\n5.funcao: define_dados_iniciais(texto_tratado) - if prefeitura_encontrada - de_para_pm \n{de_para_pm}\n\n')
        if not de_para_pm:
            de_para_pm = de_para_prefeitura.get(prefeitura_encontrada, "NAO_PROCESSAR")
            dados_iniciais_nf['de_para_pm'] = de_para_pm
            #print(de_para_pm)
    else:
        de_para_pm = "NAO_PROCESSAR"
        action_item_row_info = 'BREAK_PROCESS'
        information_row_info = 'Nao identificado dados iniciais para o documento'
        
     
        
    # Verifique cada linha do texto
    for linha in texto_tratado:
        for de_para, cnpj in templates.keys():
            if cnpj and cnpj in linha:
                cnpj_encontrado = cnpj
                dados_iniciais_nf['cnpj_encontrado'] = cnpj_encontrado
                
                
    # Saímos do loop, agora vamos verificar qual template usar
    if de_para_pm:
        template_usar = templates.get((de_para_pm, cnpj_encontrado))
        logging.info(f'usara template {template_usar} para: {cnpj_encontrado}')
        # print(template_usar)
        dados_iniciais_nf['model'] = template_usar
        if not template_usar:
            template_usar = templates.get((de_para_pm, None), "TEMPLATE_NAO_ENCONTRADO")
            dados_iniciais_nf['model'] = 'NAO_ENC.' 
            action_item_row_info = 'BREAK_PROCESS'
            information_row_info = 'model nao encontrado'
    else:
        template_usar = "TEMPLATE_NAO_ENCONTRADO"
        dados_iniciais_nf['model'] = 'NAO_ENC.'
        action_item_row_info = 'BREAK_PROCESS'
        information_row_info = 'model nao encontrado'
        
        
        
        
    
    #Confirmando se template existe em frames    
    try:        
        f_type = 'frame'
        #template_usar = 'SAO_PEDRO_SUPERMIX'
        result = filtrar_df(frames_nf_v4_df, type=f_type, de_para_pm=de_para_pm, model=template_usar)
        model = result['model'].values[0]
        if model:
            template_oficial = model
            if model == template_usar:
                dados_iniciais_nf['model'] = template_oficial
            else:    
                template_usar = "necessario cadastrar"
                dados_iniciais_nf['model'] = "CADASTRAR"
                
            dados_iniciais_nf['model'] = template_usar
        else:
            template_usar = "necessario cadastrar"
            dados_iniciais_nf['model'] = "CADASTRAR"

                
    except Exception as e:
       error_msg = (f"Erro busca do template: {e}") 
    
    dados_iniciais_nf['action_item'] = action_item_row_info 
    dados_iniciais_nf['informations'] = information_row_info         
        
    return dados_iniciais_nf  






# Dicionário para mapear palavras-chave a rótulos
mapeamento_palavras_chave = {
    "relatorio": "prov_relatorio",
    "listagem": "prov_listagem",
    "NF": "prov_nota_fiscal",
    "nf": "prov_nota_fiscal",
    "relatorio": "prov_listagem",
    "sintetico": "prov_listagem",
    "livro": "prov_livro_registro",
    "sintético": "prov_listagem",
    "nota": "prov_nota_fiscal",
    "zip": "doc_zip",
    "rar": "doc_rar",
    "valores": "prov_dinheiro",
}

# Dicionário mapeando rótulos a ações sugeridas
sugestoes_acao = {
    "prov_relatorio": "NO_PROCESS",
    "prov_listagem": "NO_PROCESS",
    "prov_nota_fiscal": "NO_PROCESS",
    "sem_rotulo": "MANUAL_REV",
    "prov_livro_registro": "NO_PROCESS",
    "doc_nao_pdf": "verificar",
    "nao_pdf": "NO_PROCESS",
    "doc_zip": "NO_PROCESS",
    "pdf_mul_paginas": "SPLIT",
}



# 2.Testando
nome_arquivo = 'batatinha_quando_nasce.pdf' # 'pre-processamento'
#palavra_chave, rotulo, acao_sugerida = define_rotulo_acao(nome_arquivo, debug)
#print(f'nome_arquivo: {nome_arquivo:>55} | palavra_chave: {palavra_chave:>20} | rotulo: {rotulo:20} | acao_sugerida: {acao_sugerida:30}')    

frames_nf_v4_df.head(5)

frames_nf_v4_df: 2.3



,id,de_para_pm,cnpj,model,version,seq,prestador,mapping_method,context_mapping,type,...,Largura,Altura,%,x0_p,y0_p,x1_p,y1_p,Largura_p,Altura_p,%_p
0,1,PM_MAGE,NaN,MAGE,2.3,1,NaN,NaN,NaN,document,...,2067.0,2923.0,NaN,0.0,0.0,600.0,760.0,600.0,760.0,NaN
1,2,PM_MAGE,NaN,MAGE,NaN,2,todos,NaN,NaN,boundaries,...,1781.0,2567.0,100.000000,NaN,NaN,NaN,NaN,0.0,0.0,100.0
2,3,PM_MAGE,NaN,MAGE,NaN,3,todos,NaN,NaN,section,...,919.0,621.0,24.191663,NaN,NaN,NaN,NaN,0.0,0.0,0.0
3,4,PM_MAGE,NaN,MAGE,NaN,4,todos,NaN,NaN,frame,...,1134.0,380.0,NaN,0.0,0.0,600.0,220.0,600.0,205.0,NaN
4,5,PM_MAGE,NaN,MAGE,NaN,5,todos,NaN,NaN,sframe_field,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN


<mark> ImportaÇao do df_root_pipe </mark>

In [4]:
# 1 Apagar os arquivos PDF:Zone
# apagar_zone(documentos_extracao_path)

batch_name = "Batch_21" #Excepcionalmente
# fake_parent_document_unique_id = generate_unique_id()

nome_formado_json = batch_name +".json"


# 0. XXX SE deseja importar o DF df_analise_pipe

df_root_pipe_path = "processamentos/df_root_analise4.xlsx"


#Le a planilha e cria df_documento_recebido
df_root_pipe = pd.read_excel(df_root_pipe_path)

# Ajusta o indice
df_root_pipe.set_index('document_unique_id', inplace=True)


df_root_pipe.head()

,seq,date_time,batch,fase_processo,nome_atividade,status_documento,acao_executada,original_file_name,directory,one_page,pages,palavra_chave,document_tag,action_item,level,parent_document_unique_id,file_hash,file_path,informations
document_unique_id,,,,,,,,,,,,,,,,,,,
923d536b-2f47-4994-8c77-538eab1c3c5f,1.0,20/09/2023 13:10:24,Batch_21,analise,scan_analise,PREPROCESS_EXTRACT,Analise,NF CRJ PRIMEIRA QUINZENA DE JULHO DE 2023.pdf,11756286.0,1.0,1.0,NF,prov_nota_fiscal,PROCESS,3.0,f976c128-1f41-4551-bffd-fac687c1c8b2,b3acfffea4847108d0064ecbd62a73359961f88741a037...,pipeline_extracao_documentos/2_documentos_para...,NaN
28303a7b-07ee-49f3-b7d6-7bc0f60baaa9,2.0,20/09/2023 13:10:24,Batch_21,analise,scan_analise,PREPROCESS_EXTRACT,Analise,Nota Fiscal Eletrônica Quallit 24072023.pdf,11766341.0,1.0,1.0,nota,prov_nota_fiscal,PROCESS,3.0,f976c128-1f41-4551-bffd-fac687c1c8b2,ae37292a66dff093838f3cde0da8bea332ec81241b1d5f...,pipeline_extracao_documentos/2_documentos_para...,NaN
e8a12fa3-90a8-4317-baaf-799014f95fb5,3.0,20/09/2023 13:10:24,Batch_21,analise,scan_analise,PREPROCESS_EXTRACT,Analise,NFE 20237.pdf,11624359.0,1.0,1.0,NF,prov_nota_fiscal,PROCESS,3.0,f976c128-1f41-4551-bffd-fac687c1c8b2,1976ffe84e27b6bb1a5840018c4a47f6bac483b10f686a...,pipeline_extracao_documentos/2_documentos_para...,NaN
44eaa169-167f-4a45-b60d-7d4f760a0664,4.0,20/09/2023 13:10:24,Batch_21,analise,scan_analise,PREPROCESS_EXTRACT,Analise,Ampla.pdf,11777624.0,1.0,1.0,default,prov_nota_fiscal,PROCESS,3.0,f976c128-1f41-4551-bffd-fac687c1c8b2,ad2522b53349fffd748376bca7fe4b90fd59359971d992...,pipeline_extracao_documentos/2_documentos_para...,NaN
80fc007c-a947-470d-923e-dad0b1ed5557,5.0,20/09/2023 13:10:24,Batch_21,analise,scan_analise,PREPROCESS_EXTRACT,Analise,Supernova.pdf,11777624.0,1.0,1.0,default,prov_nota_fiscal,PROCESS,3.0,f976c128-1f41-4551-bffd-fac687c1c8b2,c26954dad71d508d4b5315f69dc4c6291a1c5bf70548e7...,pipeline_extracao_documentos/2_documentos_para...,NaN


## <mark> <b>2.4</b> ExecuÇao do Pipeline de Extracao </mark>

In [5]:
# Esta e o principio da melhor funcao do mundo
def extracao_pipeline(qualquer_df, fase, atividade, status, debug=False, prestador=True, tomador=True, servicos=True, total=True, cnae=True, valores_impostos=True, complementares=True, outras_informacoes=True, observacoes=True):
    
    doc_info = {}
    resumo = {}
    row_teste_info = []
    time_now = cron.timenow_pt_BR()
    func_fase = fase
    func_atividade = atividade
    func_status = status
    lista_dicts = []
    conf_processo = {}
    lista_conferencia = []
   
    i = 1
    for idx, row in qualquer_df.iterrows():
        dados_iniciais = {}
        row_info = row.to_dict()
        message_erro = []
        # 1. Mapeamento de informacoes do DF
        map_document_unique_id = idx
        map_seq = row['seq']
        map_batch_name = row['batch']
        map_fase_processo = row['fase_processo']
        map_nome_atividade = row['nome_atividade']
        map_status_documento = row['status_documento']
        map_original_file_name = row['original_file_name']
        map_directory = row['directory']
        map_one_page = row['one_page']
        map_palavra_chave = row['palavra_chave']
        map_document_tag = row['document_tag']
        map_action_item = row['action_item']
        map_level = row['level']
        file_path = row['file_path']
        row_info['document_unique_id'] = map_document_unique_id
    
        # XXX Nivel 1 - Definindo que documentos serao tratados    
        if map_status_documento == 'PREPROCESS_EXTRACT':
            
            action_item_row_info = 'CONTINUE_PROCESS'
            row_info['action_item'] = action_item_row_info
            information_row_info = 'iniciado processamento'
            row_info['informations'] = information_row_info
            
            
            # 0. DADOS GERAIS DOCUMENTO
            section = "0. DADOS INICIAIS"
            try:
                valores = {}
                # 0.1. Busco prefeitura, de/para e modelo - se nao achar seta status documento para NO_PROCESS
                valores = processar_dados_iniciais(idx, row, row_info, section, map_directory, map_original_file_name, file_path, debug)
            except Exception as e:
                msg = (f'Erro ao processar_dados_iniciais: {e}')
            finally:
                row_info.update(valores)
            
            #map_status_documento_row_info = row_info.get('status_documento')
            action_item_row_info = row_info.get('action_item')
            
            # XXX Nivel 2 - Definindo que os documentos legiveis serao tratados
            if action_item_row_info == 'CONTINUE_PROCESS':
                
                prefeitura_map = row_info.get('prefeitura')
                pdf_pesquisavel_map = row_info.get('pdf_pesquisavel')
                de_para_map = row_info.get('de_para_pm')
                model_map = row_info.get('model')
                
                
                if not pdf_pesquisavel_map:
                    # NOVO PROCESSO DE TRATAMENTO DE IMAGEM - Convertendo a imagem para numpy array
                    if debug:
                        print("irei gerar a imagem_np")
                    imagem_gray, image_resized_name = convert_resize_gray(map_original_file_name, file_path, image_resized_path)
                    imagem_gray_rgb = imagem_gray.convert("RGB")
                    imagem_gray_np = np.array(imagem_gray_rgb)
                    row_info['image_np'] = imagem_gray_np
                
                # 1. CABECALHO
                # try:
                section = "1. CABECALHO"
                valores = {}
                #valores_P = {}
                f_0 = 1
                f_1 = 1
                mapping_method = "frame_&_sframe_field" # significa que as coordenadas estao em frames e os valores dos campos nos sframe_fields
                context_mapping = "data_cabecalho"
                def_replace = True 
                
                if pdf_pesquisavel_map:
                    valores = extrai_cabecalho_PDF_P(idx, row, row_info, section, pdf_pesquisavel_map, de_para_map, model_map, f_0, f_1, map_original_file_name, file_path, debug)
                    row_info.update(valores) 
                else:
                    valores = processar_cabecalho_R_PDF(idx, row, row_info, section, mapping_method, context_mapping, pdf_pesquisavel_map, model_map, map_original_file_name, file_path, debug)   
                    row_info.update(valores)
         
                #status_documento_row_info = row_info.get('status_documento')
                action_item_row_info = row_info.get('action_item')
                information_row_info = row_info.get('informations')   
                
                
                # XXX Nivel 3 - Definindo que os documentos legiveis serao tratados realmente
                if action_item_row_info == 'BREAK_PROCESS':
                    #msg = (f'Processo inicial: {map_batch_name} | {map_original_file_name} | diretorio: {map_directory} - information_row_info: {information_row_info}')
                    if debug:
                        print(f'\nINFELIZMENTE - seq: {map_seq} doc: {map_original_file_name} dir: {map_directory} - NAO SERA PROCESSADO  | inf: {information_row_info} \n\n')
               
                    #row_info['informations'] = msg
                    # logging.error(msg)
                    lista_dicts.append(row_info)
                    continue 
                
                    
                elif action_item_row_info == 'CONTINUE_PROCESS':
                    if debug:
                        print(f'\nEBA, BORA CONTINUAR - seq: {map_seq} - proxima section: | PDF Pesquisavel: {pdf_pesquisavel_map} doc: {map_original_file_name} dir: {map_directory} | action_item: {action_item_row_info} | inf: {information_row_info} \n\n')
                        print()
                        print(valores)
                    
                    information_row_info = 'Cabecalho processado'
                    row_info['informations'] = information_row_info
                    
                    
                    guarda_texto_doc = {}
                    guarda_texto_doc, linhas = cria_guarda_doc_ref_R_PDF(idx, row, de_para_map, model_map, map_original_file_name, file_path, image_resized_path, debug)
            
                    # 2. PRESTADOR DE SERVIÇO
                    if prestador == True:
                        section = "2. PRESTADOR DE SERVIÇO"
                        if debug:
                            print(f'processando {section} para: {map_original_file_name}')
                        valores = {}
                        erros_prestador = {}
                        data_tomador = {}
                        f_0 = 1
                        f_1 = 1
                        if pdf_pesquisavel_map:
                            valores = extrai_prestador_PDF_P(row, pdf_pesquisavel_map, de_para_map, model_map, f_0, f_1, map_original_file_name, file_path, debug)
                        else:
                            valores = extrai_prestador_R_PDF(idx, row, row_info, pdf_pesquisavel_map, de_para_map, model_map, f_0, f_1, map_original_file_name, file_path, debug)
                        
                        if not isinstance(valores, dict):
                            msg_erro = (f"\nErro na linha {idx}: 'valores' não é um dicionário. Tipo: {type(valores)}, Valor: {valores}")
                        else:
                            row_info.update(valores)
                            
                        # msg = (f'secao: {section:>15} processada para: {map_original_file_name} - diretorio: {map_directory}')
                        # if debug:
                        #     print(msg)
                        # logging.info(msg)
                    
                    # 3. TOMADOR DE SERVIÇO
                    if tomador == True:
                        section = "3. TOMADOR DE SERVIÇO"
                        if debug:
                            print(f'processando {section} para: {map_original_file_name}')
                        
                        valores = {}
                        erros = []
                        data_tomador = {}
                        f_0 = 1
                        f_1 = 1
                        
                        if pdf_pesquisavel_map:
                            valores = extrai_tomador_PDF_P(row, pdf_pesquisavel_map, de_para_map, model_map, f_0, f_1, map_original_file_name, file_path, debug)
                        else:   
                            valores = extrai_tomador_R_PDF(idx, row, pdf_pesquisavel_map, de_para_map, model_map, f_0, f_1, map_original_file_name, file_path, debug)
                            
                        if not isinstance(valores, dict):
                            print(f"\nErro na linha {idx}: 'valores' não é um dicionário. Tipo: {type(valores)}, Valor: {valores}")
                        else:
                            row_info.update(valores)
                        
                    # msg = (f'secao: {section:>15} processada para: {map_original_file_name} - diretorio: {map_directory}')
                    # if debug:
                    #     print(msg)
                    # logging.info(msg)
                    
                    # 4. DESCRIMINACAO DOS SERVIÇOS
                    if servicos == True:
                        if debug:
                            print(f'processando servicos para: {map_original_file_name}')
                        section = "4. DESCRIMINACAO DOS SERVIÇOS"
                        valores = {}
                        nf_data_servico = {} 
                        f_0 = 1
                        f_1 = 1
                        
                        if pdf_pesquisavel_map:
                            nf_data_servico = processar_servicos_pdf_pesquisavel(row, pdf_pesquisavel_map, model_map, map_original_file_name, file_path, debug)
                        else:
                            label = "discriminacao_servicos"
                            tipo = "field_box"
                            def_replace = True
                            
                            # ItSs  working
                            texto_extraido = extracao_documento_R_PDF(idx, row, guarda_texto_doc, section, tipo, label, de_para_map, model_map, def_replace, map_original_file_name, debug)
                            row_info[label] = texto_extraido
                            
                        msg = (f'secao: {section:>15} processada para: {map_original_file_name} - diretorio: {map_directory}')
                        if debug:
                            print(msg)
                        logging.info(msg)     


                        try:
                            texto_extraido = nf_data_servico['discriminacao_servicos'] 
                            row_info['discriminacao_servicos'] = texto_extraido 
                        except Exception as e:
                            msg = (f"doc: {map_original_file_name} | {e}")
                            discrimanacao_servico = "Descricao nao encontrada"
                            row_info['discriminacao_servicos'] = texto_extraido

                    
                    # 5. VALOR TOTAL
                    if total == True:
                        section = "5. VALOR TOTAL"
                        if debug:
                            print(f'processando {section} para: {map_original_file_name}')
                        #valores = {}
                        if pdf_pesquisavel_map:
                            valor_total_documento = processar_valor_total_PDF_P(idx, row, row_info, section, pdf_pesquisavel_map, model_map, map_original_file_name, file_path, debug)
                            if valor_total_documento:
                                if debug:
                                    print(f'\nvalor_total_documento: {valor_total_documento} | doc: {map_original_file_name}\n')
                                row_info['valor_total_nota'] = valor_total_documento
                        else:
                            label = "valor_total_nota"
                            tipo = "field_box"
                            def_replace = True
                            texto_extraido = extracao_documento_R_PDF(idx, row, guarda_texto_doc, section, tipo, label, de_para_map, model_map, def_replace, map_original_file_name, debug)
                            if texto_extraido: 
                                valor_total_match = re.search(r'R\$ ([\d,.]+)', texto_extraido)
                                if valor_total_match:
                                    valor_total_sem_formatacao = valor_total_match.group(1).replace('.', '').replace(',', '.')
                                    try:
                                        # valores['secao'] = section
                                        valor_total_documento = float(valor_total_sem_formatacao)
                                    except Exception as e:
                                        # valores['secao'] = section
                                        valor_total_documento = 0.0
                                        msg = (f'Processo inicial: {batch_name} | {map_original_file_name:>25} | diretorio: {map_directory} | {e}')
                                        #logging.error(f" {batch_name} |  doc: {original_file_name:>25} | setion:{section:20} | item: {key:>20} | erro na extracaçao | file_path: {file_path:>40} ")
                        
                                    if valor_total_documento:
                                        if debug:
                                            print(f'\nvalor_total_documento: {valor_total_documento} | doc: {map_original_file_name}\n')
                                        row_info['valor_total_nota'] = valor_total_documento
         
                    # msg = (f'secao: {section:>15} processada para: {map_original_file_name} - diretorio: {map_directory}')
                    # if debug:
                    #     print(msg)
                    # logging.info(msg)
                    
                    # 6. CNAE e Item da Lista de Serviços 
                    if cnae == True:
                        section = "6. CNAE e Item da Lista de Serviços"
                        data_box_valores = {}
                        if debug:
                            print(f'processando {section} para: {map_original_file_name}')
                        f_0_cnae = 0.95
                        f_1_cnae = 1.15
                        f_0_it = 0.95     #0.95
                        f_1_it = 1.15    # 1
                        
                        mapping_method = "frame_&_sframe_field" # significa que as coordenadas estao em frames e os valores dos campos nos sframe_fields
                        context_mapping = "data_cnae"
                        def_replace = True
                        
                        if pdf_pesquisavel_map:
                            data_box_valores = extracao_documento_CNAE_ITEM_PDF_P(idx, row, row_info, section, mapping_method, context_mapping, pdf_pesquisavel_map, model_map, def_replace, map_original_file_name, file_path, debug)
                        else:
                            data_box_valores = extracao_documento_CNAE_ITEM_R_PDF(idx, row, row_info, guarda_texto_doc, section, mapping_method, context_mapping, pdf_pesquisavel_map, model_map, def_replace, map_original_file_name, file_path, debug)
                            
                        if data_box_valores:
                            row_info.update(data_box_valores)    

                    
                    # 7. VALORES E IMPOSTOS
                    if valores_impostos == True:
                        section = "7. VALORES E IMPOSTOS"
                        # if debug:
                        print(f'processando {section} para: {map_original_file_name} - diretorio: {map_directory}')
                        valores = {}
                        nf_data_valores = {}
                        lista_impostos = []
                        f_0 = 1
                        f_1 = 1
                        if pdf_pesquisavel_map:
                            valores = extrai_valores_impostos_PDF_P(idx, row, row_info, section, pdf_pesquisavel_map, de_para_map, model_map, f_0, f_1, map_original_file_name, file_path, debug)
                            row_info.update(valores)
                        
                        else:
                            tipo = "field_box"
                            father_value = "5_frame_valores_impostos"
                            valores = extracao_impostos_R_PDF(section, tipo, father_value, de_para_map, model_map, map_original_file_name, file_path)
                            #print(valores)
                            row_info.update(valores)
                        
                    # msg = (f'secao: {section:>15} processada para: {map_original_file_name} - diretorio: {map_directory}')
                    # if debug:
                    #     print(msg)
                    # logging.info(msg) 
                    
                    # 8. DADOS COMPLEMENTARES
                    if complementares == True:
                        section = '8. DADOS COMPLEMENTARES'
                        if debug:
                            print(f'processando {section} para: {map_original_file_name}')
                        nf_data_dados_complementares = {}
                        f_0 = 1
                        f_1 = 1
                        if pdf_pesquisavel_map:
                            nf_data_valores = extrai_dados_complementares_PDF_P(idx, row, row_info, section, pdf_pesquisavel_map, de_para_map, model_map, f_0, f_1, map_original_file_name, file_path, debug)
                        else:
                            label = "dados_complementares"
                            tipo = "field_box"
                            def_replace = False
                            # ItSs  working
                            texto_extraido = extracao_complementar_R_PDF(idx, row, guarda_texto_doc, section, tipo, label, de_para_map, model_map, def_replace, map_original_file_name, debug)
                            row_info[label] = texto_extraido  

                    
                    # msg = (f'secao: {section:>15} processada para: {map_original_file_name} - diretorio: {map_directory}')
                    # if debug:
                    #     print(msg)
                    # logging.info(msg)     
                    
                    # 9. OUTRAS INFORMAÇOES / CRITICAS
                    if outras_informacoes == True:
                        section = "9. OUTRAS INFORMAÇOES / CRITICAS"
                        if debug:
                            print(f'processando {section} para: {map_original_file_name}')
                        tipo = "field_box"
                        father_value = "5_frame_inf_criticas"
                        valores = {} 
                        nf_data_outras_informacoes = {}
                        f_0 = 1
                        f_1 = 1
                        if pdf_pesquisavel_map:
                            valores = extrai_outras_informacoes_PDF_P(row, pdf_pesquisavel_map, de_para_map, model_map, f_0, f_1, map_original_file_name, file_path)
                            if valores:
                                row_info.update(valores)
                        else:
                            section = "9. OUTRAS INFORMAÇOES / CRITICAS"
                            tipo = "field_box"
                            father_value = "5_frame_inf_criticas"
                            valores = extracao_inforacoes_criticas_R_PDF(section, tipo, father_value, de_para_map, model_map, map_original_file_name, file_path)
                            if valores:
                                row_info.update(valores)
                            
                    # msg = (f'secao: {section:>15} processada para: {map_original_file_name} - diretorio: {map_directory}')
                    # if debug:
                    #     print(msg)
                    # logging.info(msg)          
                            
                    
                    # 10. OBSERVACOES
                    if observacoes == True:  
                        section = "10. OBSERVACOES"
                        if debug:
                            print(f'processando {section} para: {map_original_file_name}')  
                        data_observacao = {}
                        valores = {}
                        f_0 = 0.9
                        f_1 = 1.1
                        if pdf_pesquisavel_map:
                            valores = extrai_outras_informacoes_PDF_P(row, pdf_pesquisavel_map, de_para_map, model_map, f_0, f_1, map_original_file_name, file_path)
                            if valores:
                                row_info.update(valores)
                        else:
                            section = '10. OBSERVACOES'
                            tipo = "field_box"
                            father_value = "6_section_inf_complementares_criticas" 
                            
                            label = "observacao"
                            tipo = "field_box"
                            def_replace = True
                            valores = extracao_observacoees_R_PDF(idx, row, guarda_texto_doc, section, tipo, label, de_para_map, model_map, def_replace, map_original_file_name, debug)
                            if valores:
                                row_info.update(valores)
                    
                    # msg = (f'secao: {section:>15} processada para: {map_original_file_name} - diretorio: {map_directory}')
                    # if debug:
                    #     print(msg)           
                    
                    
                    
                lista_dicts.append(row_info)
                
                
            elif action_item_row_info == 'BREAK_PROCESS':
                
                msg = (f'Documento sem qualidade para pesquisa inicial: {map_batch_name} | {map_original_file_name} | diretorio: {map_directory}')
                row_info['informations'] = msg  
                
            
                lista_dicts.append(row_info)
                continue
                         

        
        
        
        elif map_status_documento == 'NO_PROCESS':
            msg = (f'Documento nao sera tratado neste escopo: {map_batch_name} | {map_original_file_name} | diretorio: {map_directory}')
            row_info['action_item'] = "NO_PROCESS"    
            row_info['informations'] = msg 
            lista_dicts.append(row_info)
            continue
            
            
            
        
        #lista_dicts.append(row_info)
        
        i += 1

    logging.info(f'processamento finalizado para: {batch_name}') 
    
    print(f'processamento de {i} documentos')
    
    novo_df = pd.DataFrame(lista_dicts)
    
    return novo_df

In [10]:
# XXX Processando o pipeline
ver = get_template_version('MAGE')
# analisar_pdf_pesquisavel
fase = 'analise'
atividade = 'PREPROCESS' 
status = 'PREPROCESS_EXTRACT'
raw_document_list = []
dados_prest = {}

lista_dicts = []
logging.info(f'Execuçao do pipeline para {batch_name} | df_root_pipe: {df_root_pipe_path} fase: {fase} atividade: {atividade} status: {status}  template: {ver}')


# XXX 1.Processar todas as secoes do documento
df = extracao_pipeline(df_root_pipe, fase, atividade, status, debug=False, prestador=True, tomador=True, servicos=True, total=True, cnae=True, valores_impostos=True, complementares=True, outras_informacoes=True, observacoes=True)

# 2. Processar valor Total
#df = extracao_pipeline(df_root_pipe, fase, atividade, status, debug=False, prestador=False, tomador=False, servicos=False, total=True, cnae=False, valores_impostos=False, complementares=False, outras_informacoes=False, observacoes=False)

# 3. Processar CNAE
#df = extracao_pipeline(df_root_pipe, fase, atividade, status, debug=False, prestador=False, tomador=False, servicos=False, total=False, cnae=True, valores_impostos=False, complementares=False, outras_informacoes=False, observacoes=False)

# 4. Processar Impostos
#df = extracao_pipeline(df_root_pipe, fase, atividade, status, debug=False, prestador=False, tomador=False, servicos=False, total=False, cnae=False, valores_impostos=, complementares=False, outras_informacoes=False, observacoes=False)

# 5. complementar e observaçoes
#df = extracao_pipeline(df_root_pipe, fase, atividade, status, debug=False, prestador=False, tomador=False, servicos=False, total=False, cnae=False, valores_impostos=False, complementares=True, outras_informacoes=True, observacoes=True)

0.0 500.0 600.0 700.0
processando 7. VALORES E IMPOSTOS para: NF CRJ PRIMEIRA QUINZENA DE JULHO DE 2023.pdf - diretorio: 11756286.0
0.0 500.0 600.0 700.0
processando 7. VALORES E IMPOSTOS para: Nota Fiscal Eletrônica Quallit 24072023.pdf - diretorio: 11766341.0
0.0 500.0 600.0 700.0
processando 7. VALORES E IMPOSTOS para: NFE 20237.pdf - diretorio: 11624359.0
processando 7. VALORES E IMPOSTOS para: Ampla.pdf - diretorio: 11777624.0
processando 7. VALORES E IMPOSTOS para: Supernova.pdf - diretorio: 11777624.0
processando 7. VALORES E IMPOSTOS para: Blue Lord.pdf - diretorio: 11777624.0
0.0 500.0 600.0 700.0
processando 7. VALORES E IMPOSTOS para: NF 2023158.pdf - diretorio: 11285853.0
0.0 500.0 600.0 700.0
processando 7. VALORES E IMPOSTOS para: NF 2023157.pdf - diretorio: 11285853.0
processando 7. VALORES E IMPOSTOS para: nota_07_2023.pdf - diretorio: 11778425.0
0.0 500.0 600.0 700.0
processando 7. VALORES E IMPOSTOS para: NFS-e 22.pdf - diretorio: 11778003.0
0.0 500.0 600.0 700.0
proc

In [11]:
# Ajustando DF para analises
df.set_index('document_unique_id', inplace=True)

ordem_status = ['PREPROCESS_EXTRACT', 'NO_PROCESS', 'root_analise']
ordem_action_item = ['CONTINUE_PROCESS', 'BREAK_PROCESS', 'NO_PROCESS']


df['status_documento'] = pd.Categorical(df['status_documento'], categories=ordem_status, ordered=True)
df['action_item'] = pd.Categorical(df['action_item'], categories=ordem_action_item, ordered=True)

df.sort_values(by=['status_documento', 'action_item', 'seq'], ascending=[True, True, True], inplace=True)

In [ ]:
# Criando o subset para analise
df_conf = df[['seq', 'original_file_name', 'directory', 'status_documento', 'model', 'secao', 'action_item', 'numero_nota_fiscal',  'competencia', 'dt_hr_emissao', 'codigo_verificacao', 'conf_cod', 'informations', 'cnae', 'item_lista_servicos']]
df_conf

In [12]:
# 2. XXX Criando o df_conf_export para analise excel
df_conf_export = df[['seq', 'level', 'parent_document_unique_id', 'file_hash', 'batch', 'directory', 'original_file_name', 'pages', 'one_page', 'file_path', 'palavra_chave', 'document_tag', 'action_item', 'fase_processo', 'nome_atividade', 'status_documento', 'prefeitura', 'de_para_pm', 'model', 'secao', 'pdf_pesquisavel', 'numero_nota_fiscal', 'competencia', 'dt_hr_emissao', 'codigo_verificacao', 'conf_cod', 'informations',  'p_cpf_cnpj_com_mascara', 'p_cpf_cnpj_sem_mascara','p_telefone', 'p_inscricao_estadual', 'p_inscricao_municipal', 'p_razao_social', 'p_nome_fantasia','p_endereco','p_email', 'cnae', 'item_lista_servicos','t_cpf_cnpj_com_mascara', 't_cpf_cnpj_sem_mascara','t_telefone', 't_inscricao_municipal', 't_razao_social','t_endereco','t_email', 'discriminacao_servicos', 'valor_total_nota', 'cnae', 'item_lista_servicos', 'valor_servicos', 'valor_deducao', 'desc_incond', 'base_calculo', 'aliquota', 'valor_iss', 'valor_iss_retido', 'desc_cond', 'valor_pis', 'valor_cofins', 'valor_ir', 'valor_inss', 'valor_csll', 'outras_retencoes', 'valor_liquido', 'exigibilidade_iss', 'regime_tributacao', 'simples_nacional', 'issqn_retido', 'local_pretacao_servico', 'local_incidencia', 'dados_complementares', 'observacao' ]]
df_conf_export

,seq,level,parent_document_unique_id,file_hash,batch,directory,original_file_name,pages,one_page,file_path,...,outras_retencoes,valor_liquido,exigibilidade_iss,regime_tributacao,simples_nacional,issqn_retido,local_pretacao_servico,local_incidencia,dados_complementares,observacao
document_unique_id,,,,,,,,,,,,,,,,,,,,,
923d536b-2f47-4994-8c77-538eab1c3c5f,1.0,3.0,f976c128-1f41-4551-bffd-fac687c1c8b2,b3acfffea4847108d0064ecbd62a73359961f88741a037...,Batch_21,11756286.0,NF CRJ PRIMEIRA QUINZENA DE JULHO DE 2023.pdf,1.0,1.0,pipeline_extracao_documentos/2_documentos_para...,...,0.0,7599.57,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DADOS COMPLEM ENTARES OUTRAS INFORMAÇÕES / CRI...
28303a7b-07ee-49f3-b7d6-7bc0f60baaa9,2.0,3.0,f976c128-1f41-4551-bffd-fac687c1c8b2,ae37292a66dff093838f3cde0da8bea332ec81241b1d5f...,Batch_21,11766341.0,Nota Fiscal Eletrônica Quallit 24072023.pdf,1.0,1.0,pipeline_extracao_documentos/2_documentos_para...,...,0.0,171.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://nota.pmspa.rj.gov.br 1/1 DADOS COMPLEM...
e8a12fa3-90a8-4317-baaf-799014f95fb5,3.0,3.0,f976c128-1f41-4551-bffd-fac687c1c8b2,1976ffe84e27b6bb1a5840018c4a47f6bac483b10f686a...,Batch_21,11624359.0,NFE 20237.pdf,1.0,1.0,pipeline_extracao_documentos/2_documentos_para...,...,0.0,37693.37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"DADOS COMPLEMENTARES ITAU, Agencia : 6101, Con..."
44eaa169-167f-4a45-b60d-7d4f760a0664,4.0,3.0,f976c128-1f41-4551-bffd-fac687c1c8b2,ad2522b53349fffd748376bca7fe4b90fd59359971d992...,Batch_21,11777624.0,Ampla.pdf,1.0,1.0,pipeline_extracao_documentos/2_documentos_para...,...,0.0,2750.00,None,Microempresário Individual (MEI),Sim,Sim,NaN,São Pedro da Aldeia - RJ,{'dados_complementares': ''},"Valor Aproximado dos Tributos Federais R$ 369,..."
80fc007c-a947-470d-923e-dad0b1ed5557,5.0,3.0,f976c128-1f41-4551-bffd-fac687c1c8b2,c26954dad71d508d4b5315f69dc4c6291a1c5bf70548e7...,Batch_21,11777624.0,Supernova.pdf,1.0,1.0,pipeline_extracao_documentos/2_documentos_para...,...,0.0,5000.00,None,Microempresário Individual (MEI),Sim,Sim,NaN,São Pedro da Aldeia - RJ,{'dados_complementares': ''},"Valor Aproximado dos Tributos Federais R$ 672,..."
82fa0c74-b18d-49dd-bafc-3b05ec33edc5,6.0,3.0,f976c128-1f41-4551-bffd-fac687c1c8b2,63b6603be7b1df359e67b96e9e128336b46fa3fbcfdadc...,Batch_21,11777624.0,Blue Lord.pdf,1.0,1.0,pipeline_extracao_documentos/2_documentos_para...,...,0.0,1000.00,None,Microempresário Individual (MEI),Sim,Sim,NaN,São Pedro da Aldeia - RJ,{'dados_complementares': ''},"Valor Aproximado dos Tributos Federais R$ 134,..."
6ffe74e6-64f5-421b-b1cc-f2022c3d8928,7.0,3.0,f976c128-1f41-4551-bffd-fac687c1c8b2,abcfcce04cabd261b30730f04cbb98e030f2a765bb31aa...,Batch_21,11285853.0,NF 2023158.pdf,1.0,1.0,pipeline_extracao_documentos/2_documentos_para...,...,0.0,37.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://nota.pmspa.rj.gov.br 1/1 DADOS COMPLEM...
ab93e7cb-15de-48ac-a35f-32891b339bbd,8.0,3.0,f976c128-1f41-4551-bffd-fac687c1c8b2,c8c5ae8ccca5e9befb6445e21701dc99266746e877c630...,Batch_21,11285853.0,NF 2023157.pdf,1.0,1.0,pipeline_extracao_documentos/2_documentos_para...,...,0.0,22.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://nota.pmspa.rj.gov.br 1/1 DADOS COMPLEM...
b639d369-35ac-4271-85fc-f2aed845d19f,9.0,3.0,f976c128-1f41-4551-bffd-fac687c1c8b2,33bc08587f5c227ab3e035011ed9b20340f99599e65ff7...,Batch_21,11778425.0,nota_07_2023.pdf,1.0,1.0,pipeline_extracao_documentos/2_documentos_para...,...,0.0,5070.00,Exigível,Microempresário Individual (MEI),Sim,Sim,NaN,São Pedro da Aldeia - RJ,{'dados_complementares': None},"Valor Aproximado dos Tributos Federais R$ 681,..."


In [13]:
# 3. XXX Exportando o df_conf_export para analise excel
df_conf_export.to_excel('processamentos/df_conf_export_batch_22.xlsx', index=True)

### <mark> <b>2.3.1</b> Conferencia do Processamento </mark> 

In [ ]:
# Parte2 - dados prestador - OK
df_conf2 = df[['seq', 'p_cpf_cnpj_com_mascara', 'p_cpf_cnpj_sem_mascara','p_telefone', 'p_inscricao_estadual', 'p_inscricao_municipal', 'p_razao_social', 'p_nome_fantasia','p_endereco','p_email']] #
df_conf2

In [ ]:
# Ajuste a largura máxima das colunas para um valor específico (por exemplo, 100 caracteres)
pd.set_option('display.max_colwidth', 200)


# Criando o subset para analise
df_conf = df[['seq', 'original_file_name', 'file_path']]
df_conf

In [ ]:
df['action_item'] = pd.Categorical(df['action_item'], categories=ordem_action_item, ordered=True)

In [ ]:
# Ajuste a largura da coluna 'sua_coluna' para 50 caracteres
df_conf['file_path'].astype(str).str.ljust(100), inplace=True)

In [ ]:
# Parte3 - Dados Tomador - OK
df_conf3 = df[['t_cpf_cnpj_com_mascara', 't_cpf_cnpj_sem_mascara','t_telefone', 't_inscricao_municipal', 't_razao_social','t_endereco','t_email', 'discriminacao_servicos', 'pdf_pesquisavel']] #
df_conf3

In [ ]:
# Parte 4- Outros itens
df_conf4 = df[['seq', 'original_file_name', 'pdf_pesquisavel', 'discriminacao_servicos', 'valor_total_nota', 'cnae', 'item_lista_servicos']] #
df_conf4

In [ ]:
'prefeitura', 'pdf_pesquisavel', 'image_np'

In [ ]:
df.to_excel('teste2.xlsx')
# IMPORTANTE, saber o tipo da coluna
print(df['status_documento'].dtype)

In [ ]:
df_root_pipe.head(50)

In [ ]:
df.set_index('document_unique_id', inplace=True)

In [ ]:
df_conf0 = df[['seq', 'original_file_name','directory', 'file_hash', 'file_path', 'model', 'pdf_pesquisavel','informations', 'image_np']]
df_conf0

In [ ]:
# Parte1 - dados do cabeçalho - OK
df_conf1 = df[['numero_nota_fiscal', 'competencia', 'dt_hr_emissao', 'codigo_verificacao', 'prefeitura', 'pdf_pesquisavel']]
df_conf1

In [ ]:
# Parte 4- Outros itens
df_conf4 = df[['original_file_name', 'discriminacao_servicos', 'valor_total_nota', 'cnae', 'item_lista_servicos']] #
df_conf4

In [ ]:
# Parte 5- Imposto
df_conf5 = df[['original_file_name', 'directory', 'pdf_pesquisavel', 'valor_servicos', 'valor_deducao', 'desc_incond', 'base_calculo', 'aliquota', 'valor_iss', 'valor_iss_retido', 'desc_cond', 'valor_pis', 'valor_cofins', 'valor_ir', 'valor_inss', 'valor_csll', 'outras_retencoes', 'valor_liquido', 'de_para_pm', 'batch', 'model','directory','file_path']] #
df_conf5

In [ ]:
# Parte 6- Informacoes criticas
df_conf2 = df[['original_file_name', 'pdf_pesquisavel', 'exigibilidade_iss', 'regime_tributacao', 'simples_nacional', 'issqn_retido', 'local_pretacao_servico', 'local_incidencia', 'dados_complementares', 'observacao']] #
df_conf2

In [ ]:
# DF conferencia: 
df_conferencia = df[['numero_nota_fiscal', 'codigo_verificacao', 'competencia', 'dt_hr_emissao', 'prefeitura', 'p_cpf_cnpj_com_mascara', 'p_cpf_cnpj_sem_mascara','p_telefone', 'p_inscricao_estadual', 'p_inscricao_municipal', 'p_razao_social', 'p_nome_fantasia','p_endereco','p_email', 't_cpf_cnpj_com_mascara', 't_cpf_cnpj_sem_mascara','t_telefone', 't_inscricao_municipal', 't_razao_social','t_endereco','t_email', 'discriminacao_servicos', 'valor_total_nota', 'cnae', 'item_lista_servicos','valor_servicos', 'valor_deducao', 'desc_incond', 'base_calculo', 'aliquota', 'valor_iss', 'valor_iss_retido', 'desc_cond', 'valor_pis', 'valor_cofins', 'valor_ir', 'valor_inss', 'valor_csll', 'outras_retencoes', 'valor_liquido', 'exigibilidade_iss', 'regime_tributacao', 'simples_nacional', 'issqn_retido', 'local_pretacao_servico', 'local_incidencia', 'dados_complementares', 'observacao','de_para_pm', 'batch', 'model','directory','file_path']]
df_conferencia

In [ ]:
# 1. XXX Volto novamente o indice do DF
df_conferencia.set_index('document_unique_id', inplace=True)

In [ ]:
# 10. XXX Salvo em Excel (pode ser feito durante fases)
df_conferencia.to_excel("processamento_mage_1.xlsx", index=True)

In [ ]:
# 0. XXX SE deseja importar o DF df_analise_pipe

df_conf_path = "processamento_mage_1.xlsx"


#Le a planilha e cria df_documento_recebido
df_conferencia = pd.read_excel(df_conf_path)

In [ ]:
df_processamento = pd.concat([df_conf, df_conf2, df_conf3, df_conf4,], ignore_index=False)

In [ ]:
# XXX Criando uma nova coluna no DF
df_conferencia.insert(loc=50, column='original_file_name', value=df_conferencia['file_path'].apply(lambda x: os.path.basename(x)))

In [ ]:
# 10. XXX Salvo em Excel (pode ser feito durante fases)
df_root_pipe.to_excel("df_analise_pipe_b21.xlsx", index=True)

In [ ]:
# Ajuste a largura máxima das colunas para um valor específico (por exemplo, 100 caracteres)
pd.set_option('display.max_colwidth', 100)

In [ ]:
# Ajuste a largura da coluna 'sua_coluna' para 50 caracteres
df_conf['cabecalho'] = df_conf['cabecalho'].astype(str).str.ljust(100)

### <mark> <b>2.3.2</b> ExportaÇao do Json </mark>

In [ ]:
json_file_path = os.path.join(json_path, nome_formado_json)

In [ ]:
de_para_pm

In [ ]:
arquivo_zip = "fwdnotasfaltantesnosistemadeemissoim20734_106187.zip"
titulo = (f'Processamento {batch_name} - {de_para_pm} - arq:{arquivo_zip}')

In [ ]:
# Lista para armazenar os dicionários
dados_json = {}

# Iterar sobre cada linha no DataFrame
for index, row in df_conferencia.iterrows():
    # dados_df e o dicionario para armazenar os dados da nota fiscal atual
    #diretorio = str(row['directory'])
    dados_nf = {
               "dados_NF_PDF": {
                                "data_cabecalho": {
                                    "secao": "1 - CABECALHO",
                                    "nome_prefeitura": row['prefeitura'],
                                    "numero_nota_fiscal": row['numero_nota_fiscal'],
                                    "competencia": row['competencia'],
                                    "dt_hr_emissoa": row['dt_hr_emissao'],
                                    "codigo_verificacao": row['codigo_verificacao']
                                },
                                "data_prestador": {
                                    "secao": "2. PRESTADOR DE SERVIÇO",
                                    "cpf_cnpj_com_mascara": row['p_cpf_cnpj_com_mascara'],
                                    "cpf_cnpj_sem_mascara": row['p_cpf_cnpj_sem_mascara'],
                                    "inscricao_municipal": row['p_inscricao_municipal'],
                                    "inscricao_estadual": row['p_inscricao_estadual'],
                                    "telefone": row['p_telefone'],
                                    "razao_social": row['p_razao_social'],
                                    "nome_fantasia": row['p_nome_fantasia'],
                                    "endereco": row['p_endereco'],
                                    "email": row['p_email']
                                },
                                "data_tomador": {
                                    "secao": "3. TOMADOR DE SERVIÇO",
                                    "cpf_cnpj_com_mascara": row['t_cpf_cnpj_com_mascara'],
                                    "cpf_cnpj_sem_mascara": row['t_cpf_cnpj_sem_mascara'],
                                    "rg": row['t_rg'],
                                    "inscricao_municipal": row['t_inscricao_municipal'],
                                    "inscricao_estadual": row['t_inscricao_estadual'],
                                    "telefone": row['t_telefone'],
                                    "razao_social": row['t_razao_social'],
                                    "endereco": row['t_endereco'],
                                    "email": row['t_email']
                                },
                                "data_servico": {
                                    "secao": "4. DESCRIMINACAO DOS SERVIÇOS",
                                    "discriminacao_servicos": row['discriminacao_servicos']
                                },
                                "data_valor_total": {
                                    "secao": "5. VALOR TOTAL",
                                    "valor_total_nota": row['valor_total_nota']
                                },
                                "data_CNAE": {
                                    "secao": "6. CNAE e Item da Lista de Serviços",
                                    "cnae": row['cnae'],
                                    "item_lista_servicos": row['item_lista_servicos']
                                },
                                "data_valores": {
                                    "secao": "7. VALORES E IMPOSTOS",
                                    "valor_servicos": row['valor_servicos'],
                                    "valor_deducao": row['valor_deducao'],
                                    "desc_incond" : row['desc_incond'],
                                    "base_calculo": row['base_calculo'],
                                    "aliquota": row['aliquota'],
                                    "valor_iss": row['valor_iss'],
                                    "valor_iss_retido": row['valor_iss_retido'],
                                    "desc_cond": row['desc_cond'],
                                    "valor_pis": row['valor_pis'],
                                    "valor_cofins": row['valor_cofins'],
                                    "valor_ir": row['valor_ir'],
                                    "valor_inss": row['valor_inss'],
                                    "valor_csll": row['valor_csll'],
                                    "outras_retencoes": row['outras_retencoes'],
                                    "valor_liquido": row['valor_liquido']
                                },
                                "data_dados_complementares": {
                                    "secao": "8. DADOS COMPLEMENTARES",
                                    "dados_complementares": row['dados_complementares']
                                },
                                "data_outras_informacoes": {
                                    "secao": "9. OUTRAS INFORMAÇOES / CRITICAS",
                                    "exigibilidade_iss": row['exigibilidade_iss'],
                                    "regime_tributacao": row['regime_tributacao'],
                                    "simples_nacional": row['simples_nacional'],
                                    "issqn_retido": row['issqn_retido'],
                                    "local_prestacao_servico": row['local_pretacao_servico'],
                                    "local_incidencia": row['local_incidencia']
                                },
                                "data_observacao": {
                                    "secao": "10. OBSERVACOES",
                                    "observacao": row['observacao']
                                },
                            },
                            "batch": row['batch'],    
                            "diretorio": str(row['directory']),
                            "nome_arquivo": row['original_file_name'],
                            "pdf_pesquisavel": row['pdf_pesquisavel'],
                            "modelo": row['model'],   
                            "document_unique_id": index,
                    }        
                
    
    numero_nota_fiscal = str(row['numero_nota_fiscal'])
    dados_json[numero_nota_fiscal] = dados_nf

# Salvando em formato JSON
json_file_path = os.path.join(json_path, nome_formado_json)
with open(json_file_path, 'w', encoding='utf-8') as f:
    json.dump(dados_json, f, ensure_ascii=False, indent=4)
    
print(f"As informações foram salvas em {json_file_path}")  


    

In [ ]:
df.set_index('document_unique_id', inplace=True)

### <mark> <b>2.3.3. 1</b> Desenho do Template Mathplotlib </mark>

In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle


In [ ]:
# Para testes, NAO JOGAR FORA

file_path = "pipeline_extracao_documentos/2_documentos_para_extracao/21_aguardando_processamento/Batch_21/SAO PEDRO DA ALDEIA_PDF_31282023_2257/11756286/NF CRJ PRIMEIRA QUINZENA DE JULHO DE 2023.pdf"
original_file_name = os.path.basename(file_path)

model = "SPA"
pdf_pesquisavel_map = False
mapping_method = "frame_&_sframe_field" # significa que as coordenadas estao em frames e os valores dos campos nos sframe_fields
context_mapping = "data_cabecalho"

In [ ]:
file_path = "pipeline_extracao_documentos/6_geral_administacao/temp_docs/images/processadas/ampla.jpg"

In [ ]:
image = Image.open(file_path).convert("RGB")
# Converta a imagem para um array NumPy
image_np = np.array(image)

In [ ]:
    # Determine as colunas de coordenadas a serem usadas
    x0_col, y0_col, x1_col, y1_col = ('x0_p', 'y0_p', 'x1_p', 'y1_p') if is_searchable else ('x0', 'y0', 'x1', 'y1')

In [ ]:
color_mapping = {
    "red": (1, 0, 0),
    "purple": (0.5, 0, 0.5),
    "orange": (1, 0.647, 0),
    "green": (0, 0.5, 0.196),
    "blue": (0, 0, 1),
    "yellow": (1, 1, 0),
}


def draw_boxes(image_np, df, modelo, draw_types=None):
    plt.figure(figsize=(25, 25))
    plt.imshow(image_np)
    plt.axis('off')
    
    # Filtrar baseado no modelo e nos tipos de "boxes" a serem desenhados
    filtered_df = df[df['model'] == modelo]
    if draw_types:
        filtered_df = filtered_df[filtered_df['type'].isin(draw_types)]
    
    for index, row in filtered_df.iterrows():
        x0, y0, x1, y1 = row['x0'], row['y0'], row['x1'], row['y1']
        
        color = color_mapping.get(row['color'], 'black')
        
        # Adicionando o retângulo
        plt.gca().add_patch(Rectangle((x0, y0), x1-x0, y1-y0, linewidth=1, edgecolor=color, facecolor='none'))
        
        # Adicionando o rótulo, se existir
        label = str(row['label']) if pd.notnull(row['label']) else None
        if label:
            plt.text(x0 + 10, y0 - 15, label, color=color, fontsize=10)
            plt.text(x0 + 20, y0 + 55,(x0, y0, x1, y1), color='black', fontsize=7)

    
    plt.show()


In [ ]:
nf_model_path = "pipeline_extracao_documentos/6_geral_administacao/modelos/frames_nf_v11.xlsx"


#Le a planilha e cria do DF
frames_nf_v4_df = pd.read_excel(nf_model_path)

In [ ]:
draw_boxes(image_np, frames_nf_v4_df, 'SPA', draw_types=['boundaries', 'field_box'])

In [ ]:
draw_boxes(image_np, frames_nf_v4_df, 'SPA', draw_types=['boundaries', 'frame', 'field_box'])

In [ ]:
# Modelo de apresentaçao da imagem
x0 = 0
y0 = 0
plt.figure(figsize=(25, 25))
plt.imshow(imagem_gray_np)
plt.axis('off')  # Desativa os eixos para uma visualização mais limpa
plt.text(x0 + 1, y0 + 10,original_file_name, color='black', fontsize=20)
#plt.text(original_file_name, color='black', fontsize=7)
plt.show()

In [ ]:
if mapping_method == "frame_&_sframe_field":
    tipo_4_coordinates = "frame"
    tipo_4_filter = "sframe_field"
coordinates = get_coordinates_filter_by_context(pdf_pesquisavel_map, model, context_mapping, tipo_4_coordinates)
x0, y0, x1, y1 = coordinates[0]

In [ ]:
def filtrar_df(df, **kwargs):
    query = " & ".join(f"{key} == @kwargs['{key}']" for key in kwargs)
    result = df.query(query)
    return result

# 11. Pesquiso Unique_ID por file
def get_document_id_by_file(batch, file):
    
    result = filtrar_df(df_id_relations, Batch=batch, File=file)
    document_unique_id = result['Unique_ID'].values[0]
    
    return document_unique_id

In [ ]:
df_conf0.columns

In [ ]:
df_conf0

In [ ]:
original_file_name = "Doria Marinho 0297 Raquel.pdf"

In [ ]:
result = filtrar_df(df_conf0, original_file_name=original_file_name)

In [ ]:
result

In [ ]:
# Modelo de apresentaçao da imagem
x0 = 0
y0 = 0
plt.figure(figsize=(25, 25))
plt.imshow(result['image_np'].values[0])
plt.axis('off')  # Desativa os eixos para uma visualização mais limpa
plt.text(x0 + 1, y0 + 10,original_file_name, color='black', fontsize=20)
#plt.text(original_file_name, color='black', fontsize=7)
plt.show()

In [ ]:

image_np = result['image_np'].values[0]

In [ ]:
image_np

In [ ]:
x0 = int(x0)
y0 = int(y0)
x1 = int(x1)    
y1 = int(y1)

In [ ]:
cropped_image_np = imagem_gray_np[y0:y1, x0:x1]

In [ ]:
# Modelo de apresentaçao da imagem
x0 = 0
y0 = 0
plt.figure(figsize=(5, 5))
plt.imshow(cropped_image_np)
plt.axis('off')  # Desativa os eixos para uma visualização mais limpa
#plt.text(x0 + 1, y0 + 10,original_file_name, color='black', fontsize=20)
#plt.text(original_file_name, color='black', fontsize=7)
plt.show()

In [ ]:
cropped_image_np

In [ ]:
boxes = frames_nf_v4_df[frames_nf_v4_df['type'] == 'field_box']

In [ ]:
boxes_info = {}
i = 1
for idx, row in boxes.iterrows():
    x0 = int(row['x0'])
    y0 = int(row['y0'])
    x1 = int(row['x1'])
    y1 = int(row['y1'])
    cropped_image_np = imagem_gray_np[y0:y1, x0:x1]
    
    boxes_info[f'box_{i}'] = {
        'coordinates': (x0, y0, x1, y1),
        'image': cropped_image_np,
        # ... qualquer outra informação que você deseja armazenar
    }
    i += 1


In [ ]:
boxes_info['box_1']['coordinates']

In [ ]:
boxes_info['box_1']['image']

In [ ]:
for box in boxes_info:
    x0, y0, x1, y1 = boxes_info[box]['coordinates']
    image = boxes_info[box]['image']
    # plt.figure(figsize=(25, 25))
    plt.imshow(image)
    plt.axis('off')  # Desativa os eixos para uma visualização mais limpa
    plt.show()

In [ ]:
i = 8

x0 = 0
y0 = 0
plt.figure(figsize=(10, 10))
plt.imshow(boxes_info[f'box_{i}']['image'])
plt.text(x0 , y0,boxes_info[f'box_{i}']['coordinates'], color='green', fontsize=7)
plt.axis('off')  # Desativa os eixos para uma visualização mais limpa
plt.show()


In [ ]:
# Inicialize um dicionário vazio para armazenar as informações dos "boxes"
boxes_info = {}

# Suponha que você está em um loop onde está processando vários "boxes"
for i, box in enumerate(boxes):
    # Obtenha as coordenadas do "box"
    x0, y0, x1, y1 = coordinates[0] 
    # Corte a área do "box" da imagem original
    cropped_image = imagem_gray_np[y0:y1, x0:x1]
    
    # Armazene as informações do "box" no dicionário
    boxes_info[f'box_{i}'] = {
        'coordinates': (x0, y0, x1, y1),
        'image': cropped_image,
        # ... qualquer outra informação que você deseja armazenar
    }

# Agora, `boxes_info` contém informações detalhadas sobre cada "box"


In [ ]:
import cv2
import numpy as np

# Variáveis para armazenar as coordenadas do retângulo
startX, startY, endX, endY = -1, -1, -1, -1
drawing = False

def draw_rectangle(event, x, y, flags, param):
    global startX, startY, endX, endY, drawing

    # Se o botão esquerdo do mouse for pressionado, comece a desenhar o retângulo
    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        startX, startY = x, y

    # Se o botão esquerdo do mouse for solto, finalize o retângulo
    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        endX, endY = x, y
        cv2.rectangle(img, (startX, startY), (endX, endY), (0, 255, 0), 2)

# Carregue sua imagem como um array NumPy
img = np.copy(image_np) # Substitua 'image_np' pelo seu array NumPy da imagem

# Crie uma janela e atribua a função de callback
cv2.namedWindow("Image")
cv2.setMouseCallback("Image", draw_rectangle)

while True:
    # Exibe a imagem
    cv2.imshow("Image", img)
    
    # Se a tecla 'q' for pressionada, saia do loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Fecha todas as janelas OpenCV
cv2.destroyAllWindows()


<h3><mark> <b>2.3.3.0</b> Ajustando medidas</mark></h3>

<mark> IMPORTANTE: </mark> processo de ajustar coordenadas no frames_nf_v4_df e depois salvalas 

In [ ]:
file_path = "pipeline_extracao_documentos/0_arquivos_teste_pipeline/pdf_pesquisavel/Mage/NF 6235.pdf"

In [ ]:
file_path = "pipeline_extracao_documentos/2_documentos_para_extracao/21_aguardando_processamento/Batch_20/fwdnotasfaltantesnosistemadeemissoim20734_106187/Doria Marinho 0301 Ultrascan.pdf"
original_file_name = os.path.basename(file_path)

section = "9. OUTRAS INFORMAÇOES / CRITICAS"
tipo = "field_box"
father_value = "5_frame_inf_criticas"
model_map = "MAGE"
de_para_pm = "PM_MAGE"

In [ ]:
### Foi copiada - somente para referencia
def convert_resize_gray(original_file_name, file_path, image_resized_path):

    name_image = conv_filename_no_ext(original_file_name)
    image_resized_name = os.path.join(f'{image_resized_path}/{str(name_image)}.jpg')
    pages = convert_from_path(file_path, 500, poppler_path=poppler_path)
    # 4. Verifica se ha mais que uma pagina
    if len(pages) > 1:
        raise ValueError("Erro, documento com mais de uma página")
    else:
        # 5. Iterar pelas páginas e redimensionar
        resized_pages = []
        for page in pages:
            resized_page = page.resize((4134, 5846))
            resized_pages.append(resized_page)
            
    imagem_gray = resized_pages[0].convert('L')
    imagem_gray.save(image_resized_name, 'JPEG')

    return  imagem_gray, image_resized_name

In [ ]:
# Nota de mage que nao bateu as coordenadas
file_path = "pipeline_extracao_documentos/0_arquivos_teste_pipeline/pdf_pesquisavel/Mage_fora_padrao/1128 desmonte em 01-08-23.pdf"

In [ ]:
# Processo para acertar as coordenadas de frames_nf_v4_df

novo_x0 = 700
novo_y0 = 3895
novo_x1 = 3900
novo_y1 = 4500

frames_nf_v4_df.at[indice_original, 'x0'] = novo_x0
frames_nf_v4_df.at[indice_original, 'y0'] = novo_y0
frames_nf_v4_df.at[indice_original, 'x1'] = novo_x1
frames_nf_v4_df.at[indice_original, 'y1'] = novo_y1

In [ ]:
# Para pesquisar as colunas que quero
frames_nf_v4_df[['model', 'section_json', 'label', 'reference', 'x0', 'y0', 'x1', 'y1']].head(49)

In [ ]:
frames_nf_v4_df

In [ ]:
# Path para leitura do arquivo excel do DF frames_nf_v4_df_path  (ATUALIZAR DOCUMENTO)
frames_nf_v4_df_path = "pipeline_extracao_documentos/6_geral_administacao/modelos/frames_nf_v10.xlsx"

#Le a planilha e cria do DF
frames_nf_v4_df = pd.read_excel(frames_nf_v4_df_path)

In [ ]:
# Path para leitura do arquivo excel do DF frames_nf_v4_df_path  (ATUALIZAR DOCUMENTO)
frames_nf_v4_df_path = "pipeline_extracao_documentos/6_geral_administacao/modelos/frames_nf_v10.xlsx"


# Salvando o DF para excel
frames_nf_v4_df.to_excel(frames_nf_v4_df_path, index=False)

In [ ]:
# Path para salvar o arquivo excel do DF frames_nf_v4_df_path  (ATUALIZAR DOCUMENTO)
frames_nf_v4_df_path = "pipeline_extracao_documentos/6_geral_administacao/modelos/frames_nf_v10.xlsx"

# Salvando o DF para excel
frames_nf_v4_df.to_excel(frames_nf_v4_df_path, index=False)

In [ ]:
# Listando 59 linhas do dataframe
frames_nf_v4_df.head(49)

In [ ]:
# Encontrar o índice da linha no DataFrame original
indice_original = row_frame.index[0]

# Atualizando o valor diretamente no DataFrame original
novo_valor_x0 = ...  # Substitua com o novo valor
frames_nf_v4_df.at[indice_original, 'x0'] = novo_valor_x0

# Verificando se o valor foi atualizado no DataFrame original
print(frames_nf_v4_df.at[indice_original, 'x0'])


In [ ]:
image_path = "imagem_gray.jpg"

In [ ]:
document_info = frames_nf_v4_df[frames_nf_v4_df['type'] == 'document'].iloc[0]
boundaries_info = frames_nf_v4_df[frames_nf_v4_df['type'] == 'boundaries']
sections_info = frames_nf_v4_df[frames_nf_v4_df['type'] == 'section']
frames_info = frames_nf_v4_df[frames_nf_v4_df['type'] == 'frame']
sframe_fields_info = frames_nf_v4_df[frames_nf_v4_df['type'] == 'sframe_field']
field_boxes_info = frames_nf_v4_df[frames_nf_v4_df['type'] == 'field_box']

In [ ]:
frames_nf_v4_df[frames_nf_v4_df['type'] == 'document'].iloc[0]
frames_nf_v4_df[frames_nf_v4_df['type'] == 'boundaries']
frames_nf_v4_df[frames_nf_v4_df['type'] == 'section']
frames_nf_v4_df[frames_nf_v4_df['type'] == 'frame']
frames_nf_v4_df[frames_nf_v4_df['type'] == 'field_box']

In [ ]:
field_boxes_info 

In [ ]:
frames_info

In [ ]:
model = "MAGE"
modelo = "MAGE"

In [ ]:
from PIL import Image
image = Image.open("imagem_gray.jpg")

In [ ]:
document_unique_id = result['Unique_ID'].values[0]

In [ ]:
df['coluna_cnae'] = df['coluna_cnae'].apply(lambda x: x.strip() if isinstance(x, str) else x)
